In [9]:
import csv
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model

# Define a list of dictionaries containing metadata, COBOL code, and its Java translation
cobol_java_pairs = [
    {
        "ID": 1,
        "Domain": "General",
        "Purpose": "HelloWorld",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. HelloWorld.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Hello, World!".
                            STOP RUN.""",
        "Java Translation": """public class HelloWorld {
                                  public static void main(String[] args) {
                                      System.out.println("Hello, World!");
                                  }
                              }"""
    },
    {
        "ID": 2,
        "Domain": "Arithmetic",
        "Purpose": "AddNumbers",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. AddNumbers.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 A PIC 9(2) VALUE 10.
                        01 B PIC 9(2) VALUE 20.
                        01 SUM PIC 9(4).
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE SUM = A + B.
                            DISPLAY "SUM=", SUM.
                            STOP RUN.""",
        "Java Translation": """public class AddNumbers {
                                  public static void main(String[] args) {
                                      int A = 10;
                                      int B = 20;
                                      int SUM;
                                      SUM = A + B;
                                      System.out.println("SUM=" + SUM);
                                  }
                              }"""
    },
    {
        "ID": 3,
        "Domain": "Arithmetic",
        "Purpose": "LargestNumber",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. LargestNumber.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 A PIC 9(2) VALUE 10.
                        01 B PIC 9(2) VALUE 20.
                        01 MAX PIC 9(2).
                        PROCEDURE DIVISION.
                        P1.
                            IF A > B THEN
                                MOVE A TO MAX
                            ELSE
                                MOVE B TO MAX
                            END-IF.
                            DISPLAY "MAX=", MAX.
                            STOP RUN.""",
        "Java Translation": """public class LargestNumber {
                                  public static void main(String[] args) {
                                      int A = 10;
                                      int B = 20;
                                      int MAX;
                                      if (A > B) {
                                          MAX = A;
                                      } else {
                                          MAX = B;
                                      }
                                      System.out.println("MAX=" + MAX);
                                  }
                              }"""
    },
    {
        "ID": 4,
        "Domain": "Mathematical",
        "Purpose": "Factorial",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. Factorial.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9(2) VALUE 5.
                        01 FACTORIAL PIC 9(10).
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE FACTORIAL = 1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > N
                                COMPUTE FACTORIAL = FACTORIAL * I
                            END-PERFORM.
                            DISPLAY "FACTORIAL=", FACTORIAL.
                            STOP RUN.""",
        "Java Translation": """public class Factorial {
                                  public static void main(String[] args) {
                                      int N = 5;
                                      long FACTORIAL = 1;
                                      for (int I = 1; I <= N; I++) {
                                          FACTORIAL *= I;
                                      }
                                      System.out.println("FACTORIAL=" + FACTORIAL);
                                  }
                              }"""
    },
    {
        "ID": 5,
        "Domain": "Mathematical",
        "Purpose": "FibonacciSeries",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. FibonacciSeries.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9(2) VALUE 10.
                        01 FIB1 PIC 9(10) VALUE 0.
                        01 FIB2 PIC 9(10) VALUE 1.
                        01 NEXTFIB PIC 9(10).
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY FIB1.
                            DISPLAY FIB2.
                            PERFORM UNTIL N <= 2
                                COMPUTE NEXTFIB = FIB1 + FIB2
                                DISPLAY NEXTFIB
                                COMPUTE FIB1 = FIB2
                                COMPUTE FIB2 = NEXTFIB
                                COMPUTE N = N - 1
                            END-PERFORM.
                            STOP RUN.""",
        "Java Translation": """public class FibonacciSeries {
                                  public static void main(String[] args) {
                                      int N = 10;
                                      int FIB1 = 0;
                                      int FIB2 = 1;
                                      int NEXTFIB;
                                      System.out.println(FIB1);
                                      System.out.println(FIB2);
                                      while (N > 2) {
                                          NEXTFIB = FIB1 + FIB2;
                                          System.out.println(NEXTFIB);
                                          FIB1 = FIB2;
                                          FIB2 = NEXTFIB;
                                          N--;
                                      }
                                  }
                              }"""
    },
    {
        "ID": 6,
        "Domain": "File Handling",
        "Purpose": "ReadFile",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ReadFile.
                        ENVIRONMENT DIVISION.
                        INPUT-OUTPUT SECTION.
                        FILE-CONTROL.
                            SELECT InputFile ASSIGN TO 'input.txt'.
                        DATA DIVISION.
                        FILE SECTION.
                        FD InputFile.
                        01 InputRecord PIC X(50).
                        WORKING-STORAGE SECTION.
                        01 EOF-SWITCH PIC X VALUE 'N'.
                        PROCEDURE DIVISION.
                        P1.
                            OPEN INPUT InputFile
                            READ InputFile INTO InputRecord
                            AT END MOVE 'Y' TO EOF-SWITCH
                            END-READ
                            PERFORM UNTIL EOF-SWITCH = 'Y'
                                DISPLAY InputRecord
                                READ InputFile INTO InputRecord
                                AT END MOVE 'Y' TO EOF-SWITCH
                                END-READ
                            END-PERFORM
                            CLOSE InputFile
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedReader;
                                import java.io.FileReader;
                                import java.io.IOException;

                                public class ReadFile {
                                    public static void main(String[] args) {
                                        try {
                                            BufferedReader reader = new BufferedReader(new FileReader("input.txt"));
                                            String line;
                                            while ((line = reader.readLine()) != null) {
                                                System.out.println(line);
                                            }
                                            reader.close();
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 7,
        "Domain": "String Manipulation",
        "Purpose": "ReverseString",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ReverseString.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 InputString PIC X(50) VALUE 'Hello, World!'.
                        01 OutputString PIC X(50).
                        01 I PIC 9(3) VALUE 1.
                        01 J PIC 9(3) VALUE 1.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE Function REVERSE(InputString) TO OutputString
                            DISPLAY OutputString
                            STOP RUN.
                        FUNCTION REVERSE(A) RETURNS B
                            LOCAL A, B.
                            PERFORM VARYING I FROM LENGTH OF A BY -1 UNTIL I = 0
                                MOVE FUNCTION SUBSTRING(A, I, 1) TO B(I)
                            END-PERFORM
                            SET B TO FUNCTION TRIM(B)
                        END FUNCTION.""",
        "Java Translation": """public class ReverseString {
                                    public static void main(String[] args) {
                                        String inputString = "Hello, World!";
                                        StringBuilder outputString = new StringBuilder();
                                        for (int i = inputString.length() - 1; i >= 0; i--) {
                                            outputString.append(inputString.charAt(i));
                                        }
                                        System.out.println(outputString);
                                    }
                                }"""
    },
    {
        "ID": 8,
        "Domain": "Sorting",
        "Purpose": "BubbleSort",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BubbleSort.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Array OCCURS 10 TIMES PIC 9(3) VALUE 99, 88, 77, 66, 55, 44, 33, 22, 11, 00.
                        01 N PIC 9(3) VALUE 10.
                        01 I PIC 9(3).
                        01 J PIC 9(3).
                        01 TEMP PIC 9(3).
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > N - 1
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J > N - I
                                    IF Array(J) > Array(J + 1)
                                        MOVE Array(J) TO TEMP
                                        MOVE Array(J + 1) TO Array(J)
                                        MOVE TEMP TO Array(J + 1)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            DISPLAY "Sorted Array:"
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                DISPLAY Array(I)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class BubbleSort {
                                    public static void main(String[] args) {
                                        int[] array = {99, 88, 77, 66, 55, 44, 33, 22, 11, 00};
                                        int n = array.length;
                                        int temp;
                                        for (int i = 0; i < n - 1; i++) {
                                            for (int j = 0; j < n - i - 1; j++) {
                                                if (array[j] > array[j + 1]) {
                                                    temp = array[j];
                                                    array[j] = array[j + 1];
                                                    array[j + 1] = temp;
                                                }
                                            }
                                        }
                                        System.out.println("Sorted Array:");
                                        for (int num : array) {
                                            System.out.println(num);
                                        }
                                    }
                                }"""
    },
    {
        "ID": 9,
        "Domain": "Data Structures",
        "Purpose": "Linked List",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. LinkedList.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Node.
                            02 Data PIC X(10).
                            02 NextPointer POINTER.
                        01 HeadPointer POINTER.
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INIT_LIST"
                            CALL "INSERT" USING "Data1"
                            CALL "INSERT" USING "Data2"
                            CALL "INSERT" USING "Data3"
                            CALL "DISPLAY_LIST"
                            CALL "DELETE" USING "Data2"
                            CALL "DISPLAY_LIST"
                            STOP RUN.
                        INIT_LIST.
                            SET HeadPointer TO NULL
                            EXIT PROGRAM.
                        INSERT USING Item.
                            SET Node TO ADDRESS OF Node
                            MOVE Item TO Data OF Node
                            MOVE HeadPointer TO NextPointer OF Node
                            SET HeadPointer TO Node
                            EXIT PROGRAM.
                        DISPLAY_LIST.
                            SET Node TO HeadPointer
                            PERFORM UNTIL Node = NULL
                                DISPLAY Data OF Node
                                SET Node TO NextPointer OF Node
                            END-PERFORM
                            EXIT PROGRAM.
                        DELETE USING Item.
                            SET Node TO HeadPointer
                            SET PrevNode TO NULL
                            PERFORM UNTIL Node = NULL
                                IF Data OF Node = Item
                                    IF PrevNode = NULL
                                        SET HeadPointer TO NextPointer OF Node
                                    ELSE
                                        SET NextPointer OF PrevNode TO NextPointer OF Node
                                    END-IF
                                    EXIT PROGRAM
                                END-IF
                                SET PrevNode TO Node
                                SET Node TO NextPointer OF Node
                            END-PERFORM
                            EXIT PROGRAM.""",
        "Java Translation": """public class LinkedList {
                                    static class Node {
                                        String data;
                                        Node next;

                                        Node(String data) {
                                            this.data = data;
                                            this.next = null;
                                        }
                                    }

                                    Node head;

                                    LinkedList() {
                                        this.head = null;
                                    }

                                    void insert(String item) {
                                        Node newNode = new Node(item);
                                        newNode.next = head;
                                        head = newNode;
                                    }

                                    void display() {
                                        Node current = head;
                                        while (current != null) {
                                            System.out.println(current.data);
                                            current = current.next;
                                        }
                                    }

                                    void delete(String item) {
                                        Node current = head;
                                        Node prev = null;
                                        while (current != null) {
                                            if (current.data.equals(item)) {
                                                if (prev == null) {
                                                    head = current.next;
                                                } else {
                                                    prev.next = current.next;
                                                }
                                                return;
                                            }
                                            prev = current;
                                            current = current.next;
                                        }
                                    }

                                    public static void main(String[] args) {
                                        LinkedList list = new LinkedList();
                                        list.insert("Data1");
                                        list.insert("Data2");
                                        list.insert("Data3");
                                        list.display();
                                        list.delete("Data2");
                                        list.display();
                                    }
                                }"""
    },
    {
        "ID": 10,
        "Domain": "Networking",
        "Purpose": "TCP Client",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. TCPClient.
                        ENVIRONMENT DIVISION.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Server-IP PIC X(15) VALUE "127.0.0.1".
                        01 PortNumber PIC 9(4) VALUE 8080.
                        01 Client-Socket USAGE POINTER.
                        01 Buffer PIC X(255).
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CONNECT_TO_SERVER" USING Server-IP PortNumber
                            CALL "SEND_DATA" USING "Hello, Server!"
                            CALL "RECEIVE_DATA"
                            CALL "CLOSE_CONNECTION"
                            STOP RUN.
                        CONNECT_TO_SERVER USING ServerIP PortNo.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE Client-Socket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE ServerIP, BY VALUE PortNo, BY VALUE Client-Socket
                            EXIT PROGRAM.
                        SEND_DATA USING Data.
                            STRING Data DELIMITED BY SIZE INTO Buffer
                            CALL "WSOCK32.DLL" USING BY VALUE 1, BY VALUE Buffer, BY VALUE LENGTH OF Buffer, BY VALUE Client-Socket
                            EXIT PROGRAM.
                        RECEIVE_DATA.
                            CALL "WSOCK32.DLL" USING BY VALUE 0, BY VALUE Buffer, BY VALUE LENGTH OF Buffer, BY VALUE Client-Socket
                            DISPLAY Buffer
                            EXIT PROGRAM.
                        CLOSE_CONNECTION.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY VALUE Client-Socket
                            EXIT PROGRAM.""",
        "Java Translation": """import java.io.*;
                                import java.net.*;

                                public class TCPClient {
                                    public static void main(String[] args) {
                                        try {
                                            String serverIP = "127.0.0.1";
                                            int portNumber = 8080;
                                            Socket clientSocket = new Socket(serverIP, portNumber);
                                            PrintWriter out = new PrintWriter(clientSocket.getOutputStream(), true);
                                            BufferedReader in = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));

                                            out.println("Hello, Server!");
                                            String response = in.readLine();
                                            System.out.println(response);

                                            out.close();
                                            in.close();
                                            clientSocket.close();
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 11,
        "Domain": "Database Management",
        "Purpose": "CRUD Operations",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CRUDOperations.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 EmployeeRecord.
                            02 EmployeeID PIC X(5).
                            02 EmployeeName PIC X(30).
                            02 EmployeeAge PIC 99.
                        01 DatabaseFile.
                            02 DatabaseRecord OCCURS 100 TIMES DEPENDING ON EmployeeCount.
                                03 EmployeeID PIC X(5).
                                03 EmployeeName PIC X(30).
                                03 EmployeeAge PIC 99.
                        01 EmployeeCount PIC 99 VALUE 0.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "1. Add Employee"
                            DISPLAY "2. View Employees"
                            DISPLAY "3. Update Employee"
                            DISPLAY "4. Delete Employee"
                            DISPLAY "5. Exit"
                            ACCEPT Choice
                            EVALUATE Choice
                                WHEN 1
                                    CALL "ADD_EMPLOYEE"
                                WHEN 2
                                    CALL "VIEW_EMPLOYEES"
                                WHEN 3
                                    CALL "UPDATE_EMPLOYEE"
                                WHEN 4
                                    CALL "DELETE_EMPLOYEE"
                                WHEN 5
                                    STOP RUN
                                WHEN OTHER
                                    DISPLAY "Invalid Choice"
                            END-EVALUATE
                            GO TO P1.
                        ADD_EMPLOYEE.
                            ACCEPT EmployeeID FROM STDIN
                            ACCEPT EmployeeName FROM STDIN
                            ACCEPT EmployeeAge FROM STDIN
                            ADD 1 TO EmployeeCount
                            MOVE EmployeeID TO EmployeeRecord(EmployeeCount)
                            MOVE EmployeeName TO EmployeeRecord(EmployeeCount)
                            MOVE EmployeeAge TO EmployeeRecord(EmployeeCount)
                            EXIT PROGRAM.
                        VIEW_EMPLOYEES.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > EmployeeCount
                                DISPLAY DatabaseRecord(I)
                            END-PERFORM
                            EXIT PROGRAM.
                        UPDATE_EMPLOYEE.
                            ACCEPT EmployeeID FROM STDIN
                            ACCEPT EmployeeName FROM STDIN
                            ACCEPT EmployeeAge FROM STDIN
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > EmployeeCount
                                IF EmployeeRecord(I) = EmployeeID
                                    MOVE EmployeeName TO DatabaseRecord(I)
                                    MOVE EmployeeAge TO DatabaseRecord(I)
                                    EXIT PROGRAM
                                END-IF
                            END-PERFORM
                            EXIT PROGRAM.
                        DELETE_EMPLOYEE.
                            ACCEPT EmployeeID FROM STDIN
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > EmployeeCount
                                IF EmployeeRecord(I) = EmployeeID
                                    UNSTRING DatabaseRecord(I) INTO EmployeeID, EmployeeName, EmployeeAge
                                    SUBTRACT 1 FROM EmployeeCount
                                    EXIT PROGRAM
                                END-IF
                            END-PERFORM
                            EXIT PROGRAM.""",
        "Java Translation": """import java.util.ArrayList;
                                import java.util.Scanner;

                                public class CRUDOperations {
                                    static class Employee {
                                        String employeeID;
                                        String employeeName;
                                        int employeeAge;

                                        Employee(String employeeID, String employeeName, int employeeAge) {
                                            this.employeeID = employeeID;
                                            this.employeeName = employeeName;
                                            this.employeeAge = employeeAge;
                                        }
                                    }

                                    static ArrayList<Employee> database = new ArrayList<>();

                                    public static void main(String[] args) {
                                        Scanner scanner = new Scanner(System.in);
                                        while (true) {
                                            System.out.println("1. Add Employee");
                                            System.out.println("2. View Employees");
                                            System.out.println("3. Update Employee");
                                            System.out.println("4. Delete Employee");
                                            System.out.println("5. Exit");
                                            int choice = scanner.nextInt();
                                            switch (choice) {
                                                case 1:
                                                    addEmployee();
                                                    break;
                                                case 2:
                                                    viewEmployees();
                                                    break;
                                                case 3:
                                                    updateEmployee();
                                                    break;
                                                case 4:
                                                    deleteEmployee();
                                                    break;
                                                case 5:
                                                    System.exit(0);
                                                default:
                                                    System.out.println("Invalid Choice");
                                            }
                                        }
                                    }

                                    static void addEmployee() {
                                        Scanner scanner = new Scanner(System.in);
                                        System.out.print("Enter Employee ID: ");
                                        String employeeID = scanner.nextLine();
                                        System.out.print("Enter Employee Name: ");
                                        String employeeName = scanner.nextLine();
                                        System.out.print("Enter Employee Age: ");
                                        int employeeAge = scanner.nextInt();
                                        database.add(new Employee(employeeID, employeeName, employeeAge));
                                    }

                                    static void viewEmployees() {
                                        for (Employee employee : database) {
                                            System.out.println(employee.employeeID + " | " + employee.employeeName + " | " + employee.employeeAge);
                                        }
                                    }

                                    static void updateEmployee() {
                                        Scanner scanner = new Scanner(System.in);
                                        System.out.print("Enter Employee ID to Update: ");
                                        String employeeID = scanner.nextLine();
                                        for (Employee employee : database) {
                                            if (employee.employeeID.equals(employeeID)) {
                                                System.out.print("Enter New Employee Name: ");
                                                String employeeName = scanner.nextLine();
                                                System.out.print("Enter New Employee Age: ");
                                                int employeeAge = scanner.nextInt();
                                                employee.employeeName = employeeName;
                                                employee.employeeAge = employeeAge;
                                                return;
                                            }
                                        }
                                        System.out.println("Employee ID not found.");
                                    }

                                    static void deleteEmployee() {
                                        Scanner scanner = new Scanner(System.in);
                                        System.out.print("Enter Employee ID to Delete: ");
                                        String employeeID = scanner.nextLine();
                                        for (Employee employee : database) {
                                            if (employee.employeeID.equals(employeeID)) {
                                                database.remove(employee);
                                                return;
                                            }
                                        }
                                        System.out.println("Employee ID not found.");
                                    }
                                }"""
    },
        {
        "ID": 12,
        "Domain": "Multithreading",
        "Purpose": "Banking System",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BankingSystem.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 AccountBalance PIC 99999 VALUE 10000.
                        01 WithdrawalAmount PIC 99999.
                        01 DepositAmount PIC 99999.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Current Balance: " AccountBalance
                            ACCEPT WithdrawalAmount FROM STDIN
                            IF WithdrawalAmount <= AccountBalance
                                SUBTRACT WithdrawalAmount FROM AccountBalance GIVING AccountBalance
                                DISPLAY "Withdrawal Successful"
                            ELSE
                                DISPLAY "Insufficient Funds"
                            END-IF
                            ACCEPT DepositAmount FROM STDIN
                            ADD DepositAmount TO AccountBalance GIVING AccountBalance
                            DISPLAY "Deposit Successful"
                            STOP RUN.""",
        "Java Translation": """import java.util.Scanner;

                                public class BankingSystem {
                                    static int accountBalance = 10000;

                                    public static void main(String[] args) {
                                        Scanner scanner = new Scanner(System.in);
                                        while (true) {
                                            System.out.println("Current Balance: " + accountBalance);
                                            System.out.print("Enter Withdrawal Amount: ");
                                            int withdrawalAmount = scanner.nextInt();
                                            if (withdrawalAmount <= accountBalance) {
                                                accountBalance -= withdrawalAmount;
                                                System.out.println("Withdrawal Successful");
                                            } else {
                                                System.out.println("Insufficient Funds");
                                            }
                                            System.out.print("Enter Deposit Amount: ");
                                            int depositAmount = scanner.nextInt();
                                            accountBalance += depositAmount;
                                            System.out.println("Deposit Successful");
                                        }
                                    }
                                }"""
    },
    {
        "ID": 13,
        "Domain": "Machine Learning",
        "Purpose": "Linear Regression",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. LinearRegression.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 X-Values OCCURS 10 TIMES PIC 999 VALUE 1, 2, 3, 4, 5, 6, 7, 8, 9, 10.
                        01 Y-Values OCCURS 10 TIMES PIC 999 VALUE 5, 7, 9, 11, 13, 15, 17, 19, 21, 23.
                        01 X-Mean PIC 999.
                        01 Y-Mean PIC 999.
                        01 Sum-X-X-Mean PIC 999.
                        01 Sum-Y-Y-Mean PIC 999.
                        01 Sum-X-Y PIC 999.
                        01 Sum-X-Squared PIC 999.
                        01 B1 PIC 9(3)V99.
                        01 B0 PIC 9(3)V99.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                ADD X-Values(I) TO Sum-X-X-Mean
                                ADD Y-Values(I) TO Sum-Y-Y-Mean
                            END-PERFORM
                            COMPUTE X-Mean = Sum-X-X-Mean / 10
                            COMPUTE Y-Mean = Sum-Y-Y-Mean / 10
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                COMPUTE Sum-X-Y = Sum-X-Y + ((X-Values(I) - X-Mean) * (Y-Values(I) - Y-Mean))
                                COMPUTE Sum-X-Squared = Sum-X-Squared + ((X-Values(I) - X-Mean) ** 2)
                            END-PERFORM
                            COMPUTE B1 = Sum-X-Y / Sum-X-Squared
                            COMPUTE B0 = Y-Mean - (B1 * X-Mean)
                            DISPLAY "Linear Regression Equation: Y = " B1 "X + " B0
                            STOP RUN.""",
        "Java Translation": """public class LinearRegression {
                                    public static void main(String[] args) {
                                        int[] xValues = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
                                        int[] yValues = {5, 7, 9, 11, 13, 15, 17, 19, 21, 23};
                                        int sumX = 0, sumY = 0, sumXY = 0, sumXSquared = 0;

                                        for (int i = 0; i < 10; i++) {
                                            sumX += xValues[i];
                                            sumY += yValues[i];
                                        }
                                        double xMean = sumX / 10.0;
                                        double yMean = sumY / 10.0;

                                        for (int i = 0; i < 10; i++) {
                                            sumXY += (xValues[i] - xMean) * (yValues[i] - yMean);
                                            sumXSquared += Math.pow(xValues[i] - xMean, 2);
                                        }

                                        double b1 = sumXY / sumXSquared;
                                        double b0 = yMean - (b1 * xMean);

                                        System.out.println("Linear Regression Equation: Y = " + b1 + "X + " + b0);
                                    }
                                }"""
    },
    {
        "ID": 14,
        "Domain": "Web Development",
        "Purpose": "Simple Web Server",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. SimpleWebServer.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 ServerSocket USAGE POINTER.
                        01 ClientSocket USAGE POINTER.
                        01 InStream USAGE POINTER.
                        01 OutStream USAGE POINTER.
                        01 Request PIC X(255).
                        01 Response PIC X(255).
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CREATE_SERVER" USING BY REFERENCE ServerSocket
                            PERFORM UNTIL 1 = 0
                                CALL "ACCEPT_CONNECTION" USING BY REFERENCE ServerSocket BY REFERENCE ClientSocket
                                CALL "RECEIVE_REQUEST" USING BY REFERENCE ClientSocket BY REFERENCE InStream
                                CALL "PROCESS_REQUEST" USING BY REFERENCE Request BY REFERENCE Response
                                CALL "SEND_RESPONSE" USING BY REFERENCE ClientSocket BY REFERENCE OutStream
                                CALL "CLOSE_CONNECTION" USING BY REFERENCE ClientSocket
                            END-PERFORM
                            CALL "CLOSE_SERVER" USING BY REFERENCE ServerSocket
                            STOP RUN.
                        CREATE_SERVER USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE Socket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE "127.0.0.1", BY VALUE 8080, BY REFERENCE Socket
                            EXIT PROGRAM.
                        ACCEPT_CONNECTION USING ServerSocket, ClientSocket.
                            CALL "WSOCK32.DLL" USING BY VALUE 3, BY REFERENCE ServerSocket, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        RECEIVE_REQUEST USING ClientSocket, InStream.
                            CALL "WSOCK32.DLL" USING BY VALUE 0, BY REFERENCE Request, BY VALUE LENGTH OF Request, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        PROCESS_REQUEST USING Request, Response.
                            MOVE "HTTP/1.1 200 OK" TO Response
                            STRING "Content-Type: text/html" DELIMITED BY SIZE INTO Response
                            STRING "" DELIMITED BY SIZE INTO Response
                            STRING "<html><body><h1>Hello, World!</h1></body></html>" DELIMITED BY SIZE INTO Response
                            EXIT PROGRAM.
                        SEND_RESPONSE USING ClientSocket, OutStream.
                            CALL "WSOCK32.DLL" USING BY VALUE 1, BY REFERENCE Response, BY VALUE LENGTH OF Response, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        CLOSE_CONNECTION USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.
                        CLOSE_SERVER USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.""",
        "Java Translation": """import java.io.*;
                                import java.net.*;

                                public class SimpleWebServer {
                                    public static void main(String[] args) {
                                        try {
                                            ServerSocket serverSocket = new ServerSocket(8080);
                                            System.out.println("Server started");
                                            while (true) {
                                                Socket clientSocket = serverSocket.accept();
                                                System.out.println("Client connected: " + clientSocket.getInetAddress());

                                                BufferedReader inStream = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                                String request = inStream.readLine();
                                                System.out.println("Request: " + request);

                                                OutputStream outStream = clientSocket.getOutputStream();
                                                PrintWriter out = new PrintWriter(outStream, true);

                                                out.println("HTTP/1.1 200 OK");
                                                out.println("Content-Type: text/html");
                                                out.println("");
                                                out.println("<html><body><h1>Hello, World!</h1></body></html>");

                                                out.close();
                                                inStream.close();
                                                clientSocket.close();
                                            }
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 15,
        "Domain": "Artificial Intelligence",
        "Purpose": "Chess Game",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ChessGame.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Board OCCURS 64 TIMES.
                            02 Piece PIC X(2).
                        01 Player1Piece PIC X(2) VALUE "WP".
                        01 Player2Piece PIC X(2) VALUE "BP".
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INITIALIZE_BOARD"
                            CALL "DISPLAY_BOARD"
                            CALL "MOVE" USING 7 6
                            CALL "DISPLAY_BOARD"
                            CALL "MOVE" USING 0 2
                            CALL "DISPLAY_BOARD"
                            STOP RUN.
                        INITIALIZE_BOARD.
                            MOVE " " TO Board
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 8
                                MOVE Player1Piece TO Board(I)
                                MOVE Player1Piece TO Board(8 * 1 + I)
                                MOVE Player2Piece TO Board(8 * 7 + I)
                            END-PERFORM
                            EXIT PROGRAM.
                        DISPLAY_BOARD.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 64
                                DISPLAY Board(I)
                                IF I MOD 8 = 0
                                    DISPLAY ""
                                END-IF
                            END-PERFORM
                            EXIT PROGRAM.
                        MOVE USING FromSquare ToSquare.
                            MOVE Board(FromSquare) TO Board(ToSquare)
                            MOVE " " TO Board(FromSquare)
                            EXIT PROGRAM.""",
        "Java Translation": """public class ChessGame {
                                    static String[] board = new String[64];
                                    static String player1Piece = "WP";
                                    static String player2Piece = "BP";

                                    public static void main(String[] args) {
                                        initializeBoard();
                                        displayBoard();
                                        move(7, 6);
                                        displayBoard();
                                        move(0, 2);
                                        displayBoard();
                                    }

                                    static void initializeBoard() {
                                        for (int i = 0; i < 64; i++) {
                                            board[i] = " ";
                                        }
                                        for (int i = 0; i < 8; i++) {
                                            board[i] = player1Piece;
                                            board[8 * 1 + i] = player1Piece;
                                            board[8 * 7 + i] = player2Piece;
                                        }
                                    }

                                    static void displayBoard() {
                                        for (int i = 0; i < 64; i++) {
                                            System.out.print(board[i] + " ");
                                            if ((i + 1) % 8 == 0) {
                                                System.out.println();
                                            }
                                        }
                                    }

                                    static void move(int fromSquare, int toSquare) {
                                        board[toSquare] = board[fromSquare];
                                        board[fromSquare] = " ";
                                    }
                                }"""
    },
    {
        "ID": 16,
        "Domain": "Data Structures",
        "Purpose": "Binary Search Tree",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BinarySearchTree.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 TreeNode.
                            02 Value PIC 999.
                            02 LeftChild USAGE POINTER.
                            02 RightChild USAGE POINTER.
                        01 RootNode USAGE POINTER.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                CALL "INSERT_NODE" USING I
                            END-PERFORM
                            CALL "DISPLAY_IN_ORDER" USING RootNode
                            STOP RUN.
                        INSERT_NODE USING NewValue.
                            CALL "ALLOCATE_NODE" USING NewNode
                            MOVE NewValue TO NewNode.Value
                            IF RootNode = NULL
                                MOVE NewNode TO RootNode
                            ELSE
                                PERFORM UNTIL CurrentNode = NULL
                                    MOVE CurrentNode TO ParentNode
                                    IF NewValue < CurrentNode.Value
                                        MOVE CurrentNode.LeftChild TO CurrentNode
                                    ELSE
                                        MOVE CurrentNode.RightChild TO CurrentNode
                                    END-IF
                                END-PERFORM
                                IF NewValue < ParentNode.Value
                                    MOVE NewNode TO ParentNode.LeftChild
                                ELSE
                                    MOVE NewNode TO ParentNode.RightChild
                                END-IF
                            END-IF
                            EXIT PROGRAM.
                        DISPLAY_IN_ORDER USING Node.
                            IF Node NOT = NULL
                                CALL "DISPLAY_IN_ORDER" USING Node.LeftChild
                                DISPLAY Node.Value
                                CALL "DISPLAY_IN_ORDER" USING Node.RightChild
                            END-IF
                            EXIT PROGRAM.
                        ALLOCATE_NODE USING NewNode.
                            CALL "WSOCK32.DLL" USING BY VALUE 21, BY VALUE SIZEOF TreeNode, BY REFERENCE NewNode
                            MOVE NULL TO NewNode.LeftChild
                            MOVE NULL TO NewNode.RightChild
                            EXIT PROGRAM.""",
        "Java Translation": """class TreeNode {
                                    int value;
                                    TreeNode leftChild, rightChild;

                                    TreeNode(int value) {
                                        this.value = value;
                                        leftChild = rightChild = null;
                                    }
                                }

                                public class BinarySearchTree {
                                    TreeNode root;

                                    BinarySearchTree() {
                                        root = null;
                                    }

                                    void insertNode(int newValue) {
                                        root = insertNodeRecursive(root, newValue);
                                    }

                                    TreeNode insertNodeRecursive(TreeNode root, int newValue) {
                                        if (root == null) {
                                            root = new TreeNode(newValue);
                                            return root;
                                        }
                                        if (newValue < root.value) {
                                            root.leftChild = insertNodeRecursive(root.leftChild, newValue);
                                        } else if (newValue > root.value) {
                                            root.rightChild = insertNodeRecursive(root.rightChild, newValue);
                                        }
                                        return root;
                                    }

                                    void displayInOrder(TreeNode node) {
                                        if (node != null) {
                                            displayInOrder(node.leftChild);
                                            System.out.print(node.value + " ");
                                            displayInOrder(node.rightChild);
                                        }
                                    }

                                    public static void main(String[] args) {
                                        BinarySearchTree bst = new BinarySearchTree();
                                        for (int i = 1; i <= 10; i++) {
                                            bst.insertNode(i);
                                        }
                                        bst.displayInOrder(bst.root);
                                    }
                                }"""
    },
    {
        "ID": 17,
        "Domain": "Network Programming",
        "Purpose": "TCP Client-Server Communication",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. TCPClientServer.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 ClientSocket USAGE POINTER.
                        01 ServerSocket USAGE POINTER.
                        01 InStream USAGE POINTER.
                        01 OutStream USAGE POINTER.
                        01 MessageToSend PIC X(255) VALUE "Hello, Server!".
                        01 ReceivedMessage PIC X(255).
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CREATE_SERVER" USING BY REFERENCE ServerSocket
                            CALL "CREATE_CLIENT" USING BY REFERENCE ClientSocket
                            CALL "SEND_MESSAGE" USING BY REFERENCE ClientSocket BY REFERENCE OutStream
                            CALL "RECEIVE_MESSAGE" USING BY REFERENCE ServerSocket BY REFERENCE InStream
                            CALL "CLOSE_CLIENT" USING BY REFERENCE ClientSocket
                            CALL "CLOSE_SERVER" USING BY REFERENCE ServerSocket
                            STOP RUN.
                        CREATE_SERVER USING ServerSocket.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE ServerSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE "127.0.0.1", BY VALUE 8080, BY REFERENCE ServerSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 3, BY REFERENCE ServerSocket, BY REFERENCE ServerSocket
                            EXIT PROGRAM.
                        CREATE_CLIENT USING ClientSocket.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE ClientSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE "127.0.0.1", BY VALUE 8080, BY REFERENCE ClientSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 3, BY REFERENCE ClientSocket, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        SEND_MESSAGE USING Socket, Stream.
                            CALL "WSOCK32.DLL" USING BY VALUE 1, BY REFERENCE MessageToSend, BY VALUE LENGTH OF MessageToSend, BY REFERENCE Stream
                            EXIT PROGRAM.
                        RECEIVE_MESSAGE USING Socket, Stream.
                            CALL "WSOCK32.DLL" USING BY VALUE 0, BY REFERENCE ReceivedMessage, BY VALUE LENGTH OF ReceivedMessage, BY REFERENCE Stream
                            DISPLAY ReceivedMessage
                            EXIT PROGRAM.
                        CLOSE_CLIENT USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.
                        CLOSE_SERVER USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.""",
        "Java Translation": """import java.io.*;
                                import java.net.*;

                                public class TCPClientServer {
                                    static final String SERVER_IP = "127.0.0.1";
                                    static final int SERVER_PORT = 8080;

                                    public static void main(String[] args) {
                                        try {
                                            Thread serverThread = new Thread(() -> {
                                                try {
                                                    ServerSocket serverSocket = new ServerSocket(SERVER_PORT);
                                                    Socket clientSocket = serverSocket.accept();
                                                    System.out.println("Client connected: " + clientSocket.getInetAddress());
                                                    BufferedReader inStream = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                                    String receivedMessage = inStream.readLine();
                                                    System.out.println("Received from client: " + receivedMessage);
                                                    PrintWriter outStream = new PrintWriter(clientSocket.getOutputStream(), true);
                                                    outStream.println("Hello, Client!");
                                                    inStream.close();
                                                    outStream.close();
                                                    clientSocket.close();
                                                    serverSocket.close();
                                                } catch (IOException e) {
                                                    e.printStackTrace();
                                                }
                                            });
                                            serverThread.start();

                                            Thread.sleep(1000); // Give the server some time to start

                                            Socket clientSocket = new Socket(SERVER_IP, SERVER_PORT);
                                            System.out.println("Connected to server: " + clientSocket.getInetAddress());
                                            BufferedReader inStream = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                            PrintWriter outStream = new PrintWriter(clientSocket.getOutputStream(), true);
                                            outStream.println("Hello, Server!");
                                            String receivedMessage = inStream.readLine();
                                            System.out.println("Received from server: " + receivedMessage);
                                            inStream.close();
                                            outStream.close();
                                            clientSocket.close();
                                        } catch (IOException | InterruptedException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
        {
        "ID": 18,
        "Domain": "Encryption",
        "Purpose": "RSA Algorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. RSAAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 P PIC 999 VALUE 61.
                        01 Q PIC 999 VALUE 53.
                        01 N PIC 9999 VALUE 3233.
                        01 PhiN PIC 9999 VALUE 3120.
                        01 E PIC 999 VALUE 17.
                        01 D PIC 999 VALUE 2753.
                        01 Message PIC 9999 VALUE 1234.
                        01 EncryptedMessage PIC 9999.
                        01 DecryptedMessage PIC 9999.
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE EncryptedMessage = MODEXP(Message, E, N)
                            COMPUTE DecryptedMessage = MODEXP(EncryptedMessage, D, N)
                            DISPLAY "Original Message: " Message
                            DISPLAY "Encrypted Message: " EncryptedMessage
                            DISPLAY "Decrypted Message: " DecryptedMessage
                            STOP RUN.
                        MODEXP USING Base Exponent Modulus.
                            COMPUTE Result = 1
                            PERFORM UNTIL Exponent = 0
                                IF MOD(Exponent, 2) = 1
                                    COMPUTE Result = MOD(Result * Base, Modulus)
                                END-IF
                                COMPUTE Base = MOD(Base * Base, Modulus)
                                COMPUTE Exponent = Exponent / 2
                            END-PERFORM
                            EXIT PROGRAM.""",
        "Java Translation": """public class RSAAlgorithm {
                                    public static void main(String[] args) {
                                        int p = 61;
                                        int q = 53;
                                        int n = p * q;
                                        int phiN = (p - 1) * (q - 1);
                                        int e = 17;
                                        int d = 2753;
                                        int message = 1234;
                                        int encryptedMessage = modExp(message, e, n);
                                        int decryptedMessage = modExp(encryptedMessage, d, n);
                                        System.out.println("Original Message: " + message);
                                        System.out.println("Encrypted Message: " + encryptedMessage);
                                        System.out.println("Decrypted Message: " + decryptedMessage);
                                    }

                                    static int modExp(int base, int exponent, int modulus) {
                                        int result = 1;
                                        while (exponent > 0) {
                                            if (exponent % 2 == 1) {
                                                result = (result * base) % modulus;
                                            }
                                            base = (base * base) % modulus;
                                            exponent /= 2;
                                        }
                                        return result;
                                    }
                                }"""
    },
    {
        "ID": 19,
        "Domain": "Graphics",
        "Purpose": "Simple Graphics Rendering",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. GraphicsRendering.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 ScreenWidth PIC 9999 VALUE 80.
                        01 ScreenHeight PIC 9999 VALUE 24.
                        01 ScreenBuffer OCCURS 1920 TIMES.
                            02 PixelValue PIC X(2) VALUE "  ".
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CLEAR_SCREEN"
                            CALL "DRAW_LINE" USING 0 0 79 23
                            CALL "DRAW_RECTANGLE" USING 10 5 30 15
                            CALL "DISPLAY_SCREEN"
                            STOP RUN.
                        CLEAR_SCREEN.
                            MOVE "  " TO ScreenBuffer
                            PERFORM VARYING I FROM 2 BY 2 UNTIL I > 1920
                                MOVE "  " TO ScreenBuffer(I)
                            END-PERFORM
                            EXIT PROGRAM.
                        DRAW_LINE USING X1 Y1 X2 Y2.
                            COMPUTE DeltaX = X2 - X1
                            COMPUTE DeltaY = Y2 - Y1
                            COMPUTE Slope = DeltaY / DeltaX
                            IF ABS(Slope) <= 1
                                PERFORM VARYING X FROM X1 BY 1 UNTIL X > X2
                                    COMPUTE Y = Y1 + (Slope * (X - X1))
                                    MOVE "##" TO ScreenBuffer((Y - 1) * ScreenWidth + X)
                                END-PERFORM
                            ELSE
                                COMPUTE Slope = 1 / Slope
                                PERFORM VARYING Y FROM Y1 BY 1 UNTIL Y > Y2
                                    COMPUTE X = X1 + (Slope * (Y - Y1))
                                    MOVE "##" TO ScreenBuffer((Y - 1) * ScreenWidth + X)
                                END-PERFORM
                            END-IF
                            EXIT PROGRAM.
                        DRAW_RECTANGLE USING X1 Y1 X2 Y2.
                            CALL "DRAW_LINE" USING X1 Y1 X1 Y2
                            CALL "DRAW_LINE" USING X1 Y1 X2 Y1
                            CALL "DRAW_LINE" USING X2 Y1 X2 Y2
                            CALL "DRAW_LINE" USING X1 Y2 X2 Y2
                            EXIT PROGRAM.
                        DISPLAY_SCREEN.
                            DISPLAY "┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐"
                            PERFORM VARYING Y FROM 1 BY 1 UNTIL Y > ScreenHeight
                                DISPLAY "│"
                                PERFORM VARYING X FROM 1 BY 1 UNTIL X > ScreenWidth
                                    DISPLAY ScreenBuffer((Y - 1) * ScreenWidth + X)
                                END-PERFORM
                                DISPLAY "│"
                            END-PERFORM
                            DISPLAY "└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘"
                            EXIT PROGRAM.""",
        "Java Translation": """public class GraphicsRendering {
                                    static final int SCREEN_WIDTH = 80;
                                    static final int SCREEN_HEIGHT = 24;
                                    static final String[] screenBuffer = new String[SCREEN_WIDTH * SCREEN_HEIGHT];

                                    public static void main(String[] args) {
                                        clearScreen();
                                        drawLine(0, 0, 79, 23);
                                        drawRectangle(10, 5, 30, 15);
                                        displayScreen();
                                    }

                                    static void clearScreen() {
                                        for (int i = 0; i < SCREEN_WIDTH * SCREEN_HEIGHT; i++) {
                                            screenBuffer[i] = "  ";
                                        }
                                    }

                                    static void drawLine(int x1, int y1, int x2, int y2) {
                                        int deltaX = x2 - x1;
                                        int deltaY = y2 - y1;
                                        double slope = (double) deltaY / deltaX;
                                        if (Math.abs(slope) <= 1) {
                                            for (int x = x1; x <= x2; x++) {
                                                int y = (int) (y1 + slope * (x - x1));
                                                screenBuffer[y * SCREEN_WIDTH + x] = "##";
                                            }
                                        } else {
                                            slope = 1 / slope;
                                            for (int y = y1; y <= y2; y++) {
                                                int x = (int) (x1 + slope * (y - y1));
                                                screenBuffer[y * SCREEN_WIDTH + x] = "##";
                                            }
                                        }
                                    }

                                    static void drawRectangle(int x1, int y1, int x2, int y2) {
                                        drawLine(x1, y1, x1, y2);
                                        drawLine(x1, y1, x2, y1);
                                        drawLine(x2, y1, x2, y2);
                                        drawLine(x1, y2, x2, y2);
                                    }

                                    static void displayScreen() {
                                        System.out.println("┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐");
                                        for (int y = 0; y < SCREEN_HEIGHT; y++) {
                                            System.out.print("│");
                                            for (int x = 0; x < SCREEN_WIDTH; x++) {
                                                System.out.print(screenBuffer[y * SCREEN_WIDTH + x]);
                                            }
                                            System.out.println("│");
                                        }
                                        System.out.println("└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘");
                                    }
                                }"""
    },
        {
        "ID": 20,
        "Domain": "Encryption",
        "Purpose": "Caesar Cipher",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CaesarCipher.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 PlainText PIC X(50) VALUE "HELLO".
                        01 EncryptedText PIC X(50).
                        01 Key PIC 9 VALUE 3.
                        01 I PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > LENGTH OF PlainText
                                IF PlainText(I) >= 'A' AND PlainText(I) <= 'Z'
                                    COMPUTE EncryptedText(I) = FUNCTION CHAR(UNSTRING FUNCTION UPPER-CASE(PlainText(I)) DELIMITED BY SIZE)
                                    COMPUTE EncryptedText(I) = FUNCTION CHAR(UNSTRING FUNCTION MOD(FUNCTION ORD(EncryptedText(I)) - FUNCTION ORD('A') + Key, 26) + FUNCTION ORD('A'))
                                ELSE
                                    MOVE PlainText(I) TO EncryptedText(I)
                                END-IF
                            END-PERFORM
                            DISPLAY "Encrypted Text: " EncryptedText
                            STOP RUN.""",
        "Java Translation": """public class CaesarCipher {
                                    public static void main(String[] args) {
                                        String plainText = "HELLO";
                                        StringBuilder encryptedText = new StringBuilder();
                                        int key = 3;
                                        for (char c : plainText.toCharArray()) {
                                            if (Character.isLetter(c)) {
                                                char shiftedChar = (char) (((c - 'A' + key) % 26) + 'A');
                                                encryptedText.append(Character.toUpperCase(shiftedChar));
                                            } else {
                                                encryptedText.append(c);
                                            }
                                        }
                                        System.out.println("Encrypted Text: " + encryptedText);
                                    }
                                }"""
    },
    {
        "ID": 21,
        "Domain": "Error Handling",
        "Purpose": "Exception Handling",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ExceptionHandling.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Number1 PIC 99 VALUE 10.
                        01 Number2 PIC 99 VALUE 0.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Result: "
                            DIVIDE Number1 BY Number2 GIVING Result
                                ON SIZE ERROR
                                    DISPLAY "Size Error: Cannot Divide by Zero"
                                ON ZERO
                                    DISPLAY "Zero Error: Cannot Divide by Zero"
                            END-DIVIDE
                            STOP RUN.""",
        "Java Translation": """public class ExceptionHandling {
                                    public static void main(String[] args) {
                                        int number1 = 10;
                                        int number2 = 0;
                                        try {
                                            int result = number1 / number2;
                                            System.out.println("Result: " + result);
                                        } catch (ArithmeticException e) {
                                            if (e.getMessage().equals("/ by zero")) {
                                                System.out.println("Zero Error: Cannot Divide by Zero");
                                            } else {
                                                System.out.println("Size Error: " + e.getMessage());
                                            }
                                        }
                                    }
                                }"""
    },
        {
        "ID": 22,
        "Domain": "Data Compression",
        "Purpose": "Run-Length Encoding",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. RunLengthEncoding.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 OriginalString PIC X(100) VALUE "AAABBCCDDEEEE".
                        01 EncodedString PIC X(100).
                        01 Index PIC 99 VALUE 1.
                        01 Count PIC 99.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM UNTIL Index > LENGTH OF OriginalString
                                COMPUTE Count = 1
                                PERFORM UNTIL OriginalString(Index) NOT EQUAL OriginalString(Index + 1) OR Index = LENGTH OF OriginalString
                                    ADD 1 TO Count
                                    ADD 1 TO Index
                                END-PERFORM
                                MOVE OriginalString(Index) TO EncodedString(Index)
                                MOVE Count TO EncodedString(Index + 1)
                                ADD 1 TO Index
                            END-PERFORM
                            DISPLAY "Original String: " OriginalString
                            DISPLAY "Encoded String: " EncodedString
                            STOP RUN.""",
        "Java Translation": """public class RunLengthEncoding {
                                    public static void main(String[] args) {
                                        String originalString = "AAABBCCDDEEEE";
                                        StringBuilder encodedString = new StringBuilder();
                                        int index = 0;
                                        while (index < originalString.length()) {
                                            char currentChar = originalString.charAt(index);
                                            int count = 1;
                                            while (index + count < originalString.length() && originalString.charAt(index + count) == currentChar) {
                                                count++;
                                            }
                                            encodedString.append(currentChar).append(count);
                                            index += count;
                                        }
                                        System.out.println("Original String: " + originalString);
                                        System.out.println("Encoded String: " + encodedString);
                                    }
                                }"""
    },
    {
        "ID": 23,
        "Domain": "Image Processing",
        "Purpose": "Image Resizing",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ImageResizing.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 OriginalImage PIC X(1000) VALUE "Original Image Data".
                        01 ResizedImage PIC X(500) VALUE SPACE.
                        01 ScaleFactor PIC 99V99 VALUE 0.5.
                        01 OriginalIndex PIC 999 VALUE 1.
                        01 ResizedIndex PIC 999 VALUE 1.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM UNTIL OriginalIndex > LENGTH OF OriginalImage
                                MOVE OriginalImage(OriginalIndex:OriginalIndex + 1) TO ResizedImage(ResizedIndex:ResizedIndex + 1)
                                COMPUTE OriginalIndex = OriginalIndex + 2
                                COMPUTE ResizedIndex = ResizedIndex + 1
                            END-PERFORM
                            DISPLAY "Original Image: " OriginalImage
                            DISPLAY "Resized Image: " ResizedImage
                            STOP RUN.""",
        "Java Translation": """public class ImageResizing {
                                    public static void main(String[] args) {
                                        String originalImage = "Original Image Data";
                                        StringBuilder resizedImage = new StringBuilder();
                                        double scaleFactor = 0.5;
                                        for (int i = 0; i < originalImage.length(); i += 2) {
                                            resizedImage.append(originalImage.charAt(i));
                                        }
                                        System.out.println("Original Image: " + originalImage);
                                        System.out.println("Resized Image: " + resizedImage);
                                    }
                                }"""
    },
    {
        "ID": 24,
        "Domain": "Network Programming",
        "Purpose": "Socket Programming",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. SocketProgramming.
                        ENVIRONMENT DIVISION.
                        CONFIGURATION SECTION.
                        SOURCE-COMPUTER. ALPHABETIC.
                        OBJECT-COMPUTER. ALPHABETIC.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Buffer PIC X(100) VALUE SPACE.
                        01 Port PIC 9999 VALUE 8080.
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INITIALIZE_SOCKET" USING Port
                            CALL "RECEIVE_DATA" USING Buffer
                            DISPLAY "Received Data: " Buffer
                            CALL "CLOSE_SOCKET"
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedReader;
                                import java.io.InputStreamReader;
                                import java.io.OutputStreamWriter;
                                import java.io.PrintWriter;
                                import java.net.ServerSocket;
                                import java.net.Socket;

                                public class SocketProgramming {
                                    public static void main(String[] args) {
                                        int port = 8080;
                                        try (ServerSocket serverSocket = new ServerSocket(port);
                                             Socket clientSocket = serverSocket.accept();
                                             BufferedReader in = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                             PrintWriter out = new PrintWriter(new OutputStreamWriter(clientSocket.getOutputStream()))) {
                                            StringBuilder receivedData = new StringBuilder();
                                            String line;
                                            while ((line = in.readLine()) != null) {
                                                receivedData.append(line);
                                            }
                                            System.out.println("Received Data: " + receivedData);
                                        } catch (Exception e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 25,
        "Domain": "Natural Language Processing",
        "Purpose": "Text Tokenization",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. TextTokenization.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Text PIC X(100) VALUE "This is a sample text.".
                        01 Token OCCURS 10 TIMES PIC X(10).
                        01 Delimiters PIC X(10) VALUE " ".
                        01 TokenCount PIC 99 VALUE 1.
                        PROCEDURE DIVISION.
                        P1.
                            UNSTRING Text DELIMITED BY Delimiters INTO Token(TokenCount)
                            PERFORM VARYING TokenCount FROM 1 BY 1 UNTIL Token(TokenCount) = SPACE
                                DISPLAY "Token " TokenCount ": " Token(TokenCount)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class TextTokenization {
                                    public static void main(String[] args) {
                                        String text = "This is a sample text.";
                                        String[] tokens = text.split(" ");
                                        int tokenCount = 1;
                                        for (String token : tokens) {
                                            System.out.println("Token " + tokenCount + ": " + token);
                                            tokenCount++;
                                        }
                                    }
                                }"""
    },
    {
        "ID": 26,
        "Domain": "Web Development",
        "Purpose": "REST API Development",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. RESTAPI.
                        ENVIRONMENT DIVISION.
                        CONFIGURATION SECTION.
                        SOURCE-COMPUTER. ALPHABETIC.
                        OBJECT-COMPUTER. ALPHABETIC.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Buffer PIC X(100) VALUE SPACE.
                        01 Port PIC 9999 VALUE 8080.
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INITIALIZE_SERVER" USING Port
                            CALL "LISTEN_FOR_REQUESTS"
                            CALL "SEND_RESPONSE" USING Buffer
                            DISPLAY "Response Sent: " Buffer
                            CALL "CLOSE_SERVER"
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedReader;
                                import java.io.InputStreamReader;
                                import java.io.OutputStreamWriter;
                                import java.io.PrintWriter;
                                import java.net.ServerSocket;
                                import java.net.Socket;

                                public class RESTAPI {
                                    public static void main(String[] args) {
                                        int port = 8080;
                                        try (ServerSocket serverSocket = new ServerSocket(port);
                                             Socket clientSocket = serverSocket.accept();
                                             BufferedReader in = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                             PrintWriter out = new PrintWriter(new OutputStreamWriter(clientSocket.getOutputStream()))) {
                                            String request = in.readLine();
                                            String response = "Response to " + request;
                                            out.println(response);
                                            out.flush();
                                            System.out.println("Response Sent: " + response);
                                        } catch (Exception e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 27,
        "Domain": "Robotics",
        "Purpose": "Path Planning",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. PathPlanning.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Grid OCCURS 5 TIMES PIC X(5) VALUE '00000',
                                                        '11100',
                                                        '00100',
                                                        '00111',
                                                        '11110'.
                        01 StartRow PIC 99 VALUE 1.
                        01 StartCol PIC 99 VALUE 1.
                        01 GoalRow PIC 99 VALUE 5.
                        01 GoalCol PIC 99 VALUE 5.
                        01 PathFound PIC X VALUE 'N'.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE StartRow TO CurrentRow
                            MOVE StartCol TO CurrentCol
                            PERFORM UNTIL (CurrentRow = GoalRow AND CurrentCol = GoalCol) OR PathFound = 'Y'
                                MOVE Grid(CurrentRow:CurrentCol) TO CurrentCell
                                IF CurrentRow < GoalRow AND Grid(CurrentRow + 1:CurrentCol) = '0'
                                    MOVE CurrentRow + 1 TO CurrentRow
                                ELSE IF CurrentCol < GoalCol AND Grid(CurrentRow:CurrentCol + 1) = '0'
                                    MOVE CurrentCol + 1 TO CurrentCol
                                ELSE
                                    MOVE 'Y' TO PathFound
                                END-IF
                            END-PERFORM
                            IF PathFound = 'Y'
                                DISPLAY "Path Found"
                            ELSE
                                DISPLAY "Path Not Found"
                            END-IF
                            STOP RUN.""",
        "Java Translation": """public class PathPlanning {
                                    public static void main(String[] args) {
                                        char[][] grid = {
                                            {'0', '0', '0', '0', '0'},
                                            {'1', '1', '1', '0', '0'},
                                            {'0', '0', '1', '0', '0'},
                                            {'0', '0', '1', '1', '1'},
                                            {'1', '1', '1', '1', '0'}
                                        };
                                        int startRow = 0;
                                        int startCol = 0;
                                        int goalRow = 4;
                                        int goalCol = 4;
                                        int currentRow = startRow;
                                        int currentCol = startCol;
                                        boolean pathFound = false;
                                        while ((currentRow != goalRow || currentCol != goalCol) && !pathFound) {
                                            if (currentRow < goalRow && grid[currentRow + 1][currentCol] == '0') {
                                                currentRow++;
                                            } else if (currentCol < goalCol && grid[currentRow][currentCol + 1] == '0') {
                                                currentCol++;
                                            } else {
                                                pathFound = true;
                                            }
                                        }
                                        if (pathFound) {
                                            System.out.println("Path Found");
                                        } else {
                                            System.out.println("Path Not Found");
                                        }
                                    }
                                }"""
    },
    {
        "ID": 28,
        "Domain": "Financial Analysis",
        "Purpose": "Stock Market Analysis",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. StockMarketAnalysis.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 StockPrices OCCURS 10 TIMES PIC 999V99 VALUE 100.00, 110.50, 95.75, 105.25, 112.75, 98.50, 120.00, 115.75, 125.50, 130.25.
                        01 NumPrices PIC 99 VALUE 10.
                        01 TotalPrice PIC 9999V99.
                        01 AveragePrice PIC 999V99.
                        01 MaxPrice PIC 999V99 VALUE 0.
                        01 MinPrice PIC 999V99 VALUE 999.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > NumPrices
                                COMPUTE TotalPrice = TotalPrice + StockPrices(I)
                                IF StockPrices(I) > MaxPrice
                                    MOVE StockPrices(I) TO MaxPrice
                                END-IF
                                IF StockPrices(I) < MinPrice
                                    MOVE StockPrices(I) TO MinPrice
                                END-IF
                            END-PERFORM
                            COMPUTE AveragePrice = TotalPrice / NumPrices
                            DISPLAY "Total Price: " TotalPrice
                            DISPLAY "Average Price: " AveragePrice
                            DISPLAY "Max Price: " MaxPrice
                            DISPLAY "Min Price: " MinPrice
                            STOP RUN.""",
        "Java Translation": """public class StockMarketAnalysis {
                                    public static void main(String[] args) {
                                        double[] stockPrices = {100.00, 110.50, 95.75, 105.25, 112.75, 98.50, 120.00, 115.75, 125.50, 130.25};
                                        int numPrices = 10;
                                        double totalPrice = 0;
                                        double maxPrice = 0;
                                        double minPrice = 999;
                                        for (int i = 0; i < numPrices; i++) {
                                            totalPrice += stockPrices[i];
                                            if (stockPrices[i] > maxPrice) {
                                                maxPrice = stockPrices[i];
                                            }
                                            if (stockPrices[i] < minPrice) {
                                                minPrice = stockPrices[i];
                                            }
                                        }
                                        double averagePrice = totalPrice / numPrices;
                                        System.out.println("Total Price: " + totalPrice);
                                        System.out.println("Average Price: " + averagePrice);
                                        System.out.println("Max Price: " + maxPrice);
                                        System.out.println("Min Price: " + minPrice);
                                    }
                                }"""
    },
    {
        "ID": 29,
        "Domain": "Graph Theory",
        "Purpose": "Shortest Path",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ShortestPath.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Graph OCCURS 5 TIMES PIC X(5) VALUE '00000',
                                                            '11100',
                                                            '00100',
                                                            '00111',
                                                            '11110'.
                        01 StartNode PIC 99 VALUE 1.
                        01 GoalNode PIC 99 VALUE 5.
                        01 VisitedNodes OCCURS 5 TIMES PIC X VALUE 'N'.
                        01 Queue OCCURS 10 TIMES PIC 99.
                        01 Front PIC 99 VALUE 1.
                        01 Rear PIC 99 VALUE 1.
                        01 PathFound PIC X VALUE 'N'.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE StartNode TO Queue(Rear)
                            MOVE 'Y' TO VisitedNodes(StartNode)
                            PERFORM UNTIL Front > Rear OR PathFound = 'Y'
                                IF Queue(Front) = GoalNode
                                    MOVE 'Y' TO PathFound
                                ELSE
                                    PERFORM VARYING I FROM 1 BY 1 UNTIL I > 5
                                        IF Graph(Queue(Front):I) = '1' AND VisitedNodes(I) = 'N'
                                            COMPUTE Rear = Rear + 1
                                            MOVE I TO Queue(Rear)
                                            MOVE 'Y' TO VisitedNodes(I)
                                        END-IF
                                    END-PERFORM
                                    COMPUTE Front = Front + 1
                                END-IF
                            END-PERFORM
                            IF PathFound = 'Y'
                                DISPLAY "Shortest Path Found"
                            ELSE
                                DISPLAY "Shortest Path Not Found"
                            END-IF
                            STOP RUN.""",
        "Java Translation": """public class ShortestPath {
                                    public static void main(String[] args) {
                                        char[][] graph = {
                                            {'0', '0', '0', '0', '0'},
                                            {'1', '1', '1', '0', '0'},
                                            {'0', '0', '1', '0', '0'},
                                            {'0', '0', '1', '1', '1'},
                                            {'1', '1', '1', '1', '0'}
                                        };
                                        int startNode = 1;
                                        int goalNode = 5;
                                        boolean[] visitedNodes = new boolean[5];
                                        int[] queue = new int[10];
                                        int front = 0;
                                        int rear = 0;
                                        boolean pathFound = false;
                                        queue[rear] = startNode;
                                        rear++;
                                        visitedNodes[startNode] = true;
                                        while (front < rear && !pathFound) {
                                            if (queue[front] == goalNode) {
                                                pathFound = true;
                                            } else {
                                                for (int i = 0; i < 5; i++) {
                                                    if (graph[queue[front]][i] == '1' && !visitedNodes[i]) {
                                                        queue[rear] = i;
                                                        rear++;
                                                        visitedNodes[i] = true;
                                                    }
                                                }
                                                front++;
                                            }
                                        }
                                        if (pathFound) {
                                            System.out.println("Shortest Path Found");
                                        } else {
                                            System.out.println("Shortest Path Not Found");
                                        }
                                    }
                                }"""
    },
    {
        "ID": 30,
        "Domain": "Distributed Systems",
        "Purpose": "MapReduce",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. MapReduce.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 InputData PIC X(100) VALUE "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.".
                        01 OutputData PIC X(100) VALUE SPACE.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM UNTIL InputData = SPACE
                                UNSTRING InputData DELIMITED BY SPACE INTO OutputData
                                DISPLAY "Word: " OutputData
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class MapReduce {
                                    public static void main(String[] args) {
                                        String inputData = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.";
                                        String[] words = inputData.split(" ");
                                        for (String word : words) {
                                            System.out.println("Word: " + word);
                                        }
                                    }
                                }"""
    },
    {
        "ID": 31,
        "Domain": "Blockchain",
        "Purpose": "Cryptocurrency Implementation",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. Cryptocurrency.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 TransactionHistory OCCURS 10 TIMES.
                            05 Sender PIC X(10) VALUE "Alice".
                            05 Receiver PIC X(10) VALUE "Bob".
                            05 Amount PIC 999V99 VALUE 10.50.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                DISPLAY "Sender: " TransactionHistory(Sender)
                                DISPLAY "Receiver: " TransactionHistory(Receiver)
                                DISPLAY "Amount: " TransactionHistory(Amount)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class Cryptocurrency {
                                    public static void main(String[] args) {
                                        String[] sender = {"Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice"};
                                        String[] receiver = {"Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob"};
                                        double[] amount = {10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50};
                                        for (int i = 0; i < 10; i++) {
                                            System.out.println("Sender: " + sender[i]);
                                            System.out.println("Receiver: " + receiver[i]);
                                            System.out.println("Amount: " + amount[i]);
                                        }
                                    }
                                }"""
    },
    {
        "ID": 32,
        "Domain": "Augmented Reality",
        "Purpose": "Object Detection",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ObjectDetection.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 CameraFrame PIC X(1000) VALUE "Camera frame data".
                        01 DetectedObjects OCCURS 10 TIMES.
                            05 ObjectName PIC X(10).
                            05 ObjectLocationX PIC 999.
                            05 ObjectLocationY PIC 999.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                DISPLAY "Object Name: " DetectedObjects(ObjectName)
                                DISPLAY "Object Location X: " DetectedObjects(ObjectLocationX)
                                DISPLAY "Object Location Y: " DetectedObjects(ObjectLocationY)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class ObjectDetection {
                                    public static void main(String[] args) {
                                        String cameraFrame = "Camera frame data";
                                        String[] objectName = new String[10];
                                        int[] objectLocationX = new int[10];
                                        int[] objectLocationY = new int[10];
                                        for (int i = 0; i < 10; i++) {
                                            System.out.println("Object Name: " + objectName[i]);
                                            System.out.println("Object Location X: " + objectLocationX[i]);
                                            System.out.println("Object Location Y: " + objectLocationY[i]);
                                        }
                                    }
                                }"""
    },
        {
        "ID": 4,
        "Domain": "File Handling",
        "Purpose": "ReadFile",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ReadFile.
                        ENVIRONMENT DIVISION.
                        INPUT-OUTPUT SECTION.
                        FILE-CONTROL.
                        SELECT InputFile ASSIGN TO "input.txt"
                            ORGANIZATION IS LINE SEQUENTIAL.
                        DATA DIVISION.
                        FILE SECTION.
                        FD InputFile.
                        01 InputRecord PIC X(50).
                        WORKING-STORAGE SECTION.
                        01 EOF-SWITCH PIC X VALUE 'N'.
                        PROCEDURE DIVISION.
                        P1.
                            OPEN INPUT InputFile.
                            READ InputFile INTO InputRecord
                                AT END MOVE 'Y' TO EOF-SWITCH.
                            DISPLAY "RECORD READ:", InputRecord.
                            CLOSE InputFile.
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedReader;
                                import java.io.FileReader;
                                import java.io.IOException;

                                public class ReadFile {
                                    public static void main(String[] args) {
                                        try {
                                            BufferedReader br = new BufferedReader(new FileReader("input.txt"));
                                            String line;
                                            while ((line = br.readLine()) != null) {
                                                System.out.println("RECORD READ:" + line);
                                            }
                                            br.close();
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 5,
        "Domain": "File Handling",
        "Purpose": "WriteFile",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. WriteFile.
                        ENVIRONMENT DIVISION.
                        INPUT-OUTPUT SECTION.
                        FILE-CONTROL.
                        SELECT OutputFile ASSIGN TO "output.txt"
                            ORGANIZATION IS LINE SEQUENTIAL.
                        DATA DIVISION.
                        FILE SECTION.
                        FD OutputFile.
                        01 OutputRecord PIC X(50) VALUE "Hello, World!".
                        WORKING-STORAGE SECTION.
                        PROCEDURE DIVISION.
                        P1.
                            OPEN OUTPUT OutputFile.
                            WRITE OutputRecord FROM OutputRecord.
                            CLOSE OutputFile.
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedWriter;
                                import java.io.FileWriter;
                                import java.io.IOException;

                                public class WriteFile {
                                    public static void main(String[] args) {
                                        try {
                                            BufferedWriter bw = new BufferedWriter(new FileWriter("output.txt"));
                                            bw.write("Hello, World!");
                                            bw.close();
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
        {
        "ID": 7,
        "Domain": "Basic",
        "Purpose": "HelloName",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. HelloName.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NAME PIC X(20) VALUE "Alice".
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Hello, " NAME "!".
                            STOP RUN.""",
        "Java Translation": """public class HelloName {
                                  public static void main(String[] args) {
                                      String NAME = "Alice";
                                      System.out.println("Hello, " + NAME + "!");
                                  }
                              }"""
    },
    {
        "ID": 8,
        "Domain": "Basic",
        "Purpose": "CalculateArea",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateArea.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 LENGTH PIC 9(2) VALUE 10.
                        01 WIDTH PIC 9(2) VALUE 5.
                        01 AREA PIC 9(4).
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE AREA = LENGTH * WIDTH.
                            DISPLAY "AREA=", AREA.
                            STOP RUN.""",
        "Java Translation": """public class CalculateArea {
                                  public static void main(String[] args) {
                                      int LENGTH = 10;
                                      int WIDTH = 5;
                                      int AREA;
                                      AREA = LENGTH * WIDTH;
                                      System.out.println("AREA=" + AREA);
                                  }
                              }"""
    },
        {
        "ID": 9,
        "Domain": "Basic",
        "Purpose": "CheckEvenOdd",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckEvenOdd.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9(2) VALUE 5.
                        PROCEDURE DIVISION.
                        P1.
                            IF NUM MOD 2 = 0
                                DISPLAY NUM " is even."
                            ELSE
                                DISPLAY NUM " is odd."
                            STOP RUN.""",
        "Java Translation": """public class CheckEvenOdd {
                                  public static void main(String[] args) {
                                      int NUM = 5;
                                      if (NUM % 2 == 0) {
                                          System.out.println(NUM + " is even.");
                                      } else {
                                          System.out.println(NUM + " is odd.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 10,
        "Domain": "Basic",
        "Purpose": "CheckPositiveNegative",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckPositiveNegative.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC S9(2) VALUE -5.
                        PROCEDURE DIVISION.
                        P1.
                            IF NUM >= 0
                                DISPLAY NUM " is positive or zero."
                            ELSE
                                DISPLAY NUM " is negative."
                            STOP RUN.""",
        "Java Translation": """public class CheckPositiveNegative {
                                  public static void main(String[] args) {
                                      int NUM = -5;
                                      if (NUM >= 0) {
                                          System.out.println(NUM + " is positive or zero.");
                                      } else {
                                          System.out.println(NUM + " is negative.");
                                      }
                                  }
                              }"""
    },
        {
        "ID": 12,
        "Domain": "Basic",
        "Purpose": "SwapNumbers",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. SwapNumbers.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 A PIC 9(2) VALUE 5.
                        01 B PIC 9(2) VALUE 10.
                        01 TEMP PIC 9(2).
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Before swapping, A=", A, " B=", B.
                            MOVE A TO TEMP.
                            MOVE B TO A.
                            MOVE TEMP TO B.
                            DISPLAY "After swapping, A=", A, " B=", B.
                            STOP RUN.""",
        "Java Translation": """public class SwapNumbers {
                                  public static void main(String[] args) {
                                      int A = 5;
                                      int B = 10;
                                      int TEMP;
                                      System.out.println("Before swapping, A=" + A + " B=" + B + ".");
                                      TEMP = A;
                                      A = B;
                                      B = TEMP;
                                      System.out.println("After swapping, A=" + A + " B=" + B + ".");
                                  }
                              }"""
    },
    {
        "ID": 13,
        "Domain": "Basic",
        "Purpose": "CalculateAverage",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateAverage.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM1 PIC 9(2) VALUE 10.
                        01 NUM2 PIC 9(2) VALUE 20.
                        01 NUM3 PIC 9(2) VALUE 30.
                        01 AVERAGE PIC 9(3)V9(2).
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE AVERAGE = (NUM1 + NUM2 + NUM3) / 3.
                            DISPLAY "AVERAGE=", AVERAGE.
                            STOP RUN.""",
        "Java Translation": """public class CalculateAverage {
                                  public static void main(String[] args) {
                                      int NUM1 = 10;
                                      int NUM2 = 20;
                                      int NUM3 = 30;
                                      double AVERAGE;
                                      AVERAGE = (NUM1 + NUM2 + NUM3) / 3.0;
                                      System.out.println("AVERAGE=" + AVERAGE);
                                  }
                              }"""
    },
    {
        "ID": 14,
        "Domain": "Basic",
        "Purpose": "CheckLeapYear",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckLeapYear.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 YEAR PIC 9(4) VALUE 2024.
                        PROCEDURE DIVISION.
                        P1.
                            IF ((YEAR MOD 4 = 0 AND YEAR MOD 100 != 0) OR YEAR MOD 400 = 0)
                                DISPLAY YEAR " is a leap year."
                            ELSE
                                DISPLAY YEAR " is not a leap year."
                            STOP RUN.""",
        "Java Translation": """public class CheckLeapYear {
                                  public static void main(String[] args) {
                                      int YEAR = 2024;
                                      if ((YEAR % 4 == 0 && YEAR % 100 != 0) || YEAR % 400 == 0) {
                                          System.out.println(YEAR + " is a leap year.");
                                      } else {
                                          System.out.println(YEAR + " is not a leap year.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 15,
        "Domain": "Basic",
        "Purpose": "CalculatePower",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculatePower.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 BASE PIC 9(2) VALUE 2.
                        01 EXPONENT PIC 9(2) VALUE 3.
                        01 RESULT PIC 9(4).
                        PROCEDURE DIVISION.
                        P1.
                            SET RESULT TO BASE ** EXPONENT.
                            DISPLAY "RESULT=", RESULT.
                            STOP RUN.""",
        "Java Translation": """public class CalculatePower {
                                  public static void main(String[] args) {
                                      int BASE = 2;
                                      int EXPONENT = 3;
                                      int RESULT;
                                      RESULT = (int) Math.pow(BASE, EXPONENT);
                                      System.out.println("RESULT=" + RESULT);
                                  }
                              }"""
    },
    {
        "ID": 16,
        "Domain": "Basic",
        "Purpose": "CheckPalindrome",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckPalindrome.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9(5) VALUE 12321.
                        01 REVERSED PIC 9(5).
                        01 TEMP PIC 9(5).
                        PROCEDURE DIVISION.
                        P1.
                            MOVE NUM TO TEMP.
                            PERFORM REVERSE-NUMBER.
                            IF TEMP = REVERSED
                                DISPLAY NUM " is a palindrome."
                            ELSE
                                DISPLAY NUM " is not a palindrome."
                            STOP RUN.
                        REVERSE-NUMBER.
                            SET REVERSED TO 0.
                            PERFORM UNTIL TEMP = 0
                                DIVIDE TEMP BY 10 GIVING TEMP REMAINDER REVERSE-DIGIT
                                MULTIPLY REVERSED BY 10
                                ADD REVERSE-DIGIT TO REVERSED
                                DIVIDE TEMP BY 10 GIVING TEMP
                            END-PERFORM.""",
        "Java Translation": """public class CheckPalindrome {
                                  public static void main(String[] args) {
                                      int NUM = 12321;
                                      int REVERSED = 0;
                                      int TEMP = NUM;
                                      while (TEMP != 0) {
                                          int REMAINDER = TEMP % 10;
                                          REVERSED = REVERSED * 10 + REMAINDER;
                                          TEMP = TEMP / 10;
                                      }
                                      if (NUM == REVERSED) {
                                          System.out.println(NUM + " is a palindrome.");
                                      } else {
                                          System.out.println(NUM + " is not a palindrome.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 17,
        "Domain": "Basic",
        "Purpose": "CalculateSimpleInterest",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateSimpleInterest.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 PRINCIPAL PIC 9(5)V99 VALUE 1000.50.
                        01 RATE PIC 9(2)V9(2) VALUE 5.5.
                        01 TIME PIC 9(1) VALUE 2.
                        01 SIMPLE-INTEREST PIC 9(5)V99.
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE SIMPLE-INTEREST = (PRINCIPAL * RATE * TIME) / 100.
                            DISPLAY "Simple Interest=", SIMPLE-INTEREST.
                            STOP RUN.""",
        "Java Translation": """public class CalculateSimpleInterest {
                                  public static void main(String[] args) {
                                      double PRINCIPAL = 1000.50;
                                      double RATE = 5.5;
                                      int TIME = 2;
                                      double SIMPLE_INTEREST;
                                      SIMPLE_INTEREST = (PRINCIPAL * RATE * TIME) / 100;
                                      System.out.println("Simple Interest=" + SIMPLE_INTEREST);
                                  }
                              }"""
    },
    {
        "ID": 18,
        "Domain": "Basic",
        "Purpose": "CalculateCompoundInterest",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateCompoundInterest.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 PRINCIPAL PIC 9(5)V99 VALUE 1000.50.
                        01 RATE PIC 9(2)V9(2) VALUE 5.5.
                        01 TIME PIC 9(1) VALUE 2.
                        01 COMPOUND-INTEREST PIC 9(5)V99.
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE COMPOUND-INTEREST = PRINCIPAL * (1 + RATE / 100) ** TIME - PRINCIPAL.
                            DISPLAY "Compound Interest=", COMPOUND-INTEREST.
                            STOP RUN.""",
        "Java Translation": """public class CalculateCompoundInterest {
                                  public static void main(String[] args) {
                                      double PRINCIPAL = 1000.50;
                                      double RATE = 5.5;
                                      int TIME = 2;
                                      double COMPOUND_INTEREST;
                                      COMPOUND_INTEREST = PRINCIPAL * (Math.pow((1 + RATE / 100), TIME)) - PRINCIPAL;
                                      System.out.println("Compound Interest=" + COMPOUND_INTEREST);
                                  }
                              }"""
    },
    {
        "ID": 19,
        "Domain": "Basic",
        "Purpose": "CheckPrimeNumber",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckPrimeNumber.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9(2) VALUE 17.
                        01 IS-PRIME PIC X VALUE 'Y'.
                        01 I PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            IF NUM <= 1
                                MOVE 'N' TO IS-PRIME
                            ELSE
                                PERFORM CHECK-PRIME
                            IF IS-PRIME = 'Y'
                                DISPLAY NUM " is a prime number."
                            ELSE
                                DISPLAY NUM " is not a prime number."
                            STOP RUN.
                        CHECK-PRIME.
                            DIVIDE NUM BY 2 GIVING I REMAINDER 0
                            PERFORM VARYING I FROM 2 BY 1 UNTIL I >= NUM / 2
                                IF NUM MOD I = 0
                                    MOVE 'N' TO IS-PRIME
                                    EXIT PERFORM
                            END-PERFORM.""",
        "Java Translation": """public class CheckPrimeNumber {
                                  public static void main(String[] args) {
                                      int NUM = 17;
                                      boolean IS_PRIME = true;
                                      if (NUM <= 1) {
                                          IS_PRIME = false;
                                      } else {
                                          for (int i = 2; i <= NUM / 2; i++) {
                                              if (NUM % i == 0) {
                                                  IS_PRIME = false;
                                                  break;
                                              }
                                          }
                                      }
                                      if (IS_PRIME) {
                                          System.out.println(NUM + " is a prime number.");
                                      } else {
                                          System.out.println(NUM + " is not a prime number.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 20,
        "Domain": "Basic",
        "Purpose": "CheckVowelConsonant",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckVowelConsonant.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                         01 CHAR PIC X VALUE 'A'.
                        PROCEDURE DIVISION.
                        P1.
                            IF CHAR = 'A' OR CHAR = 'E' OR CHAR = 'I' OR CHAR = 'O' OR CHAR = 'U'
                                DISPLAY CHAR " is a vowel."
                            ELSE
                                DISPLAY CHAR " is a consonant."
                            STOP RUN.""",
        "Java Translation": """public class CheckVowelConsonant {
                                  public static void main(String[] args) {
                                      char CHAR = 'A';
                                      if (CHAR == 'A' || CHAR == 'E' || CHAR == 'I' || CHAR == 'O' || CHAR == 'U') {
                                          System.out.println(CHAR + " is a vowel.");
                                      } else {
                                          System.out.println(CHAR + " is a consonant.");
                                      }
                                  }
                              }"""
    },
        {
        "ID": 21,
        "Domain": "Basic",
        "Purpose": "CheckPositiveNegativeZero",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckPositiveNegativeZero.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC S9(2) VALUE -5.
                        PROCEDURE DIVISION.
                        P1.
                            IF NUM > 0
                                DISPLAY NUM " is positive."
                            ELSE IF NUM < 0
                                DISPLAY NUM " is negative."
                            ELSE
                                DISPLAY NUM " is zero."
                            STOP RUN.""",
        "Java Translation": """public class CheckPositiveNegativeZero {
                                  public static void main(String[] args) {
                                      int NUM = -5;
                                      if (NUM > 0) {
                                          System.out.println(NUM + " is positive.");
                                      } else if (NUM < 0) {
                                          System.out.println(NUM + " is negative.");
                                      } else {
                                          System.out.println(NUM + " is zero.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 22,
        "Domain": "Basic",
        "Purpose": "CalculateSumOfDigits",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateSumOfDigits.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9(5) VALUE 12345.
                        01 SUM PIC 9(5).
                        01 TEMP PIC 9(5).
                        01 REMAINDER PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE NUM TO TEMP.
                            SET SUM TO 0.
                            PERFORM UNTIL TEMP = 0
                                DIVIDE TEMP BY 10 GIVING TEMP REMAINDER REMAINDER
                                ADD REMAINDER TO SUM
                            END-PERFORM.
                            DISPLAY "SUM=", SUM.
                            STOP RUN.""",
        "Java Translation": """public class CalculateSumOfDigits {
                                  public static void main(String[] args) {
                                      int NUM = 12345;
                                      int SUM = 0;
                                      int TEMP = NUM;
                                      int REMAINDER;
                                      while (TEMP != 0) {
                                          REMAINDER = TEMP % 10;
                                          SUM += REMAINDER;
                                          TEMP /= 10;
                                      }
                                      System.out.println("SUM=" + SUM);
                                  }
                              }"""
    },
    {
        "ID": 23,
        "Domain": "Basic",
        "Purpose": "CheckArmstrongNumber",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckArmstrongNumber.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9(5) VALUE 153.
                        01 TEMP PIC 9(5).
                        01 REMAINDER PIC 9.
                        01 SUM PIC 9(5).
                        PROCEDURE DIVISION.
                        P1.
                            MOVE NUM TO TEMP.
                            SET SUM TO 0.
                            PERFORM UNTIL TEMP = 0
                                DIVIDE TEMP BY 10 GIVING TEMP REMAINDER REMAINDER
                                COMPUTE SUM = SUM + REMAINDER ** 3
                            END-PERFORM.
                            IF SUM = NUM
                                DISPLAY NUM " is an Armstrong number."
                            ELSE
                                DISPLAY NUM " is not an Armstrong number."
                            STOP RUN.""",
        "Java Translation": """public class CheckArmstrongNumber {
                                  public static void main(String[] args) {
                                      int NUM = 153;
                                      int TEMP = NUM;
                                      int REMAINDER;
                                      int SUM = 0;
                                      while (TEMP != 0) {
                                          REMAINDER = TEMP % 10;
                                          SUM += Math.pow(REMAINDER, 3);
                                          TEMP /= 10;
                                      }
                                      if (SUM == NUM) {
                                          System.out.println(NUM + " is an Armstrong number.");
                                      } else {
                                          System.out.println(NUM + " is not an Armstrong number.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 24,
        "Domain": "Basic",
        "Purpose": "PrintPattern",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. PrintPattern.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 I PIC 9 VALUE 1.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM UNTIL I > 5
                                DISPLAY "*" TIMES I
                                ADD 1 TO I
                            END-PERFORM.
                            STOP RUN.""",
        "Java Translation": """public class PrintPattern {
                                  public static void main(String[] args) {
                                      for (int i = 1; i <= 5; i++) {
                                          for (int j = 1; j <= i; j++) {
                                              System.out.print("*");
                                          }
                                          System.out.println();
                                      }
                                  }
                              }"""
    },
    {
        "ID": 25,
        "Domain": "Basic",
        "Purpose": "FindLargestAmongThreeNumbers",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. FindLargestAmongThreeNumbers.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 A PIC 9(2) VALUE 10.
                        01 B PIC 9(2) VALUE 20.
                        01 C PIC 9(2) VALUE 15.
                        PROCEDURE DIVISION.
                        P1.
                            IF A > B AND A > C
                                DISPLAY "Largest number is A=", A
                            ELSE IF B > A AND B > C
                                DISPLAY "Largest number is B=", B
                            ELSE
                                DISPLAY "Largest number is C=", C
                            STOP RUN.""",
        "Java Translation": """public class FindLargestAmongThreeNumbers {
                                  public static void main(String[] args) {
                                      int A = 10;
                                      int B = 20;
                                      int C = 15;
                                      if (A > B && A > C) {
                                          System.out.println("Largest number is A=" + A);
                                      } else if (B > A && B > C) {
                                          System.out.println("Largest number is B=" + B);
                                      } else {
                                          System.out.println("Largest number is C=" + C);
                                      }
                                  }
                              }"""
    },
    {
        "ID": 26,
        "Domain": "Basic",
        "Purpose": "CalculateGCD",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateGCD.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM1 PIC 9(2) VALUE 24.
                        01 NUM2 PIC 9(2) VALUE 36.
                        01 GCD PIC 9(2).
                        01 TEMP PIC 9(2).
                        PROCEDURE DIVISION.
                        P1.
                            IF NUM1 < NUM2
                                MOVE NUM1 TO TEMP
                                MOVE NUM2 TO NUM1
                                MOVE TEMP TO NUM2
                            PERFORM CALCULATE-GCD
                            DISPLAY "GCD of " NUM1 " and " NUM2 " is " GCD
                            STOP RUN.
                        CALCULATE-GCD.
                            PERFORM UNTIL NUM2 = 0
                                DIVIDE NUM1 BY NUM2 GIVING TEMP REMAINDER
                                MOVE NUM2 TO NUM1
                                MOVE TEMP TO NUM2
                            END-PERFORM.
                            MOVE NUM1 TO GCD.""",
        "Java Translation": """public class CalculateGCD {
                                  public static void main(String[] args) {
                                      int NUM1 = 24;
                                      int NUM2 = 36;
                                      int GCD, TEMP;
                                      if (NUM1 < NUM2) {
                                          TEMP = NUM1;
                                          NUM1 = NUM2;
                                          NUM2 = TEMP;
                                      }
                                      while (NUM2 != 0) {
                                          TEMP = NUM2;
                                          NUM2 = NUM1 % NUM2;
                                          NUM1 = TEMP;
                                      }
                                      GCD = NUM1;
                                      System.out.println("GCD of " + NUM1 + " and " + NUM2 + " is " + GCD);
                                  }
                              }"""
    },
    {
        "ID": 27,
        "Domain": "Basic",
        "Purpose": "ReverseNumber",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ReverseNumber.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9(5) VALUE 12345.
                        01 REVERSED PIC 9(5).
                        01 TEMP PIC 9(5).
                        01 REMAINDER PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE NUM TO TEMP.
                            SET REVERSED TO 0.
                            PERFORM UNTIL TEMP = 0
                                DIVIDE TEMP BY 10 GIVING TEMP REMAINDER REMAINDER
                                MULTIPLY REVERSED BY 10
                                ADD REMAINDER TO REVERSED
                            END-PERFORM.
                            DISPLAY "REVERSED=", REVERSED.
                            STOP RUN.""",
        "Java Translation": """public class ReverseNumber {
                                  public static void main(String[] args) {
                                      int NUM = 12345;
                                      int REVERSED = 0;
                                      int TEMP = NUM;
                                      int REMAINDER;
                                      while (TEMP != 0) {
                                          REMAINDER = TEMP % 10;
                                          REVERSED = REVERSED * 10 + REMAINDER;
                                          TEMP /= 10;
                                      }
                                      System.out.println("REVERSED=" + REVERSED);
                                  }
                              }"""
    },
    {
        "ID": 28,
        "Domain": "Basic",
        "Purpose": "CalculateFibonacciSeries",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateFibonacciSeries.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 10.
                        01 A PIC 9 VALUE 0.
                        01 B PIC 9 VALUE 1.
                        01 SUM PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Fibonacci Series:"
                            DISPLAY A
                            DISPLAY B
                            PERFORM VARYING I FROM 3 BY 1 UNTIL I > N
                                COMPUTE SUM = A + B
                                DISPLAY SUM
                                MOVE B TO A
                                MOVE SUM TO B
                            END-PERFORM.
                            STOP RUN.""",
        "Java Translation": """public class CalculateFibonacciSeries {
                                  public static void main(String[] args) {
                                      int N = 10;
                                      int A = 0;
                                      int B = 1;
                                      int SUM;
                                      System.out.println("Fibonacci Series:");
                                      System.out.println(A);
                                      System.out.println(B);
                                      for (int i = 3; i <= N; i++) {
                                          SUM = A + B;
                                          System.out.println(SUM);
                                          A = B;
                                          B = SUM;
                                      }
                                  }
                              }"""
    },
    {
        "ID": 29,
        "Domain": "Basic",
        "Purpose": "CheckPositiveInteger",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckPositiveInteger.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9(2) VALUE 5.
                        PROCEDURE DIVISION.
                        P1.
                            IF NUM > 0
                                DISPLAY NUM " is a positive integer."
                            ELSE
                                DISPLAY NUM " is not a positive integer."
                            STOP RUN.""",
        "Java Translation": """public class CheckPositiveInteger {
                                  public static void main(String[] args) {
                                      int NUM = 5;
                                      if (NUM > 0) {
                                          System.out.println(NUM + " is a positive integer.");
                                      } else {
                                          System.out.println(NUM + " is not a positive integer.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 30,
        "Domain": "Basic",
        "Purpose": "CalculateFactorial",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateFactorial.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 FACTORIAL PIC 9(5).
                        PROCEDURE DIVISION.
                        P1.
                            SET FACTORIAL TO 1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > N
                                MULTIPLY FACTORIAL BY I
                            END-PERFORM.
                            DISPLAY "Factorial of " N " is " FACTORIAL
                            STOP RUN.""",
        "Java Translation": """public class CalculateFactorial {
                                  public static void main(String[] args) {
                                      int N = 5;
                                      int FACTORIAL = 1;
                                      for (int i = 1; i <= N; i++) {
                                          FACTORIAL *= i;
                                      }
                                      System.out.println("Factorial of " + N + " is " + FACTORIAL);
                                  }
                              }"""
    },
        {
        "ID": 31,
        "Domain": "Basic",
        "Purpose": "CheckLeapYear",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CheckLeapYear.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 YEAR PIC 9(4) VALUE 2024.
                        PROCEDURE DIVISION.
                        P1.
                            IF YEAR MOD 4 = 0 AND (YEAR MOD 100 <> 0 OR YEAR MOD 400 = 0)
                                DISPLAY YEAR " is a leap year."
                            ELSE
                                DISPLAY YEAR " is not a leap year."
                            STOP RUN.""",
        "Java Translation": """public class CheckLeapYear {
                                  public static void main(String[] args) {
                                      int YEAR = 2024;
                                      if (YEAR % 4 == 0 && (YEAR % 100 != 0 || YEAR % 400 == 0)) {
                                          System.out.println(YEAR + " is a leap year.");
                                      } else {
                                          System.out.println(YEAR + " is not a leap year.");
                                      }
                                  }
                              }"""
    },
    {
        "ID": 32,
        "Domain": "Basic",
        "Purpose": "PrintTable",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. PrintTable.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM PIC 9 VALUE 5.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                DISPLAY NUM " * " I " = " NUM * I
                            END-PERFORM.
                            STOP RUN.""",
        "Java Translation": """public class PrintTable {
                                  public static void main(String[] args) {
                                      int NUM = 5;
                                      for (int i = 1; i <= 10; i++) {
                                          System.out.println(NUM + " * " + i + " = " + (NUM * i));
                                      }
                                  }
                              }"""
    },
    {
        "ID": 33,
        "Domain": "Basic",
        "Purpose": "CalculateAverage",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateAverage.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 NUM1 PIC 9(2) VALUE 10.
                        01 NUM2 PIC 9(2) VALUE 20.
                        01 NUM3 PIC 9(2) VALUE 30.
                        01 AVG PIC 9(3).
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE AVG = (NUM1 + NUM2 + NUM3) / 3
                            DISPLAY "Average=", AVG
                            STOP RUN.""",
        "Java Translation": """public class CalculateAverage {
                                  public static void main(String[] args) {
                                      int NUM1 = 10;
                                      int NUM2 = 20;
                                      int NUM3 = 30;
                                      int AVG;
                                      AVG = (NUM1 + NUM2 + NUM3) / 3;
                                      System.out.println("Average=" + AVG);
                                  }
                              }"""
    },
    {
        "ID": 34,
        "Domain": "Basic",
        "Purpose": "CalculatePowerUsingRecursion",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculatePowerUsingRecursion.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 BASE PIC 9 VALUE 2.
                        01 EXPONENT PIC 9 VALUE 3.
                        01 RESULT PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE RESULT = FUNCTION POWER(BASE, EXPONENT)
                            DISPLAY "RESULT=", RESULT
                            STOP RUN.
                        FUNCTION POWER
                            USING B E
                            RETURNING P.
                            IF E = 0
                                MOVE 1 TO P
                            ELSE
                                COMPUTE P = B * FUNCTION POWER(B, E - 1)
                            END-IF.""",
        "Java Translation": """public class CalculatePowerUsingRecursion {
                                  public static void main(String[] args) {
                                      int BASE = 2;
                                      int EXPONENT = 3;
                                      int RESULT;
                                      RESULT = power(BASE, EXPONENT);
                                      System.out.println("RESULT=" + RESULT);
                                  }

                                  public static int power(int BASE, int EXPONENT) {
                                      if (EXPONENT == 0) {
                                          return 1;
                                      } else {
                                          return BASE * power(BASE, EXPONENT - 1);
                                      }
                                  }
                              }"""
    },
    {
        "ID": 35,
        "Domain": "Basic",
        "Purpose": "CalculateSumOfNaturalNumbers",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CalculateSumOfNaturalNumbers.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 SUM PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE SUM = N * (N + 1) / 2
                            DISPLAY "Sum of first " N " natural numbers=", SUM
                            STOP RUN.""",
        "Java Translation": """public class CalculateSumOfNaturalNumbers {
                                  public static void main(String[] args) {
                                      int N = 5;
                                      int SUM;
                                      SUM = N * (N + 1) / 2;
                                      System.out.println("Sum of first " + N + " natural numbers=" + SUM);
                                  }
                              }"""
    },
    {
        "ID": 84,
        "Domain": "Advanced",
        "Purpose": "DepthFirstSearch",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. DepthFirstSearch.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 ADJACENCY-MATRIX.
                            05  ROW OCCURS 5 TIMES.
                                10  COLUMN PIC 9 OCCURS 5 TIMES.
                        01 STACK PIC 9(2) OCCURS 5 TIMES.
                        01 TOP PIC 9 VALUE 0.
                        01 START-NODE PIC 9 VALUE 1.
                        01 VISITED PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 I PIC 9.
                        01 J PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE 0 TO TOP
                            PERFORM UNTIL TOP = 0
                                MOVE STACK(TOP) TO I
                                MOVE 'Y' TO VISITED(I)
                                DISPLAY "Node visited: " I
                                MOVE TOP - 1 TO TOP
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J > N
                                    IF ADJACENCY-MATRIX(I, J) = 1 AND VISITED(J) = 'N'
                                        ADD 1 TO TOP
                                        MOVE J TO STACK(TOP)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """import java.util.Stack;

                              public class DepthFirstSearch {
                                  private static final int N = 5;
                                  private static int[][] adjacencyMatrix = new int[N + 1][N + 1];
                                  private static boolean[] visited = new boolean[N + 1];
                                  private static Stack<Integer> stack = new Stack<>();

                                  public static void main(String[] args) {
                                      int startNode = 1;

                                      // Initialize adjacency matrix (1-based indexing)
                                      // adjacencyMatrix[i][j] = 1 if there is an edge from i to j

                                      depthFirstSearch(startNode);
                                  }

                                  private static void depthFirstSearch(int startNode) {
                                      stack.push(startNode);
                                      visited[startNode] = true;

                                      while (!stack.isEmpty()) {
                                          int node = stack.pop();
                                          System.out.println("Node visited: " + node);

                                          for (int j = 1; j <= N; j++) {
                                              if (adjacencyMatrix[node][j] == 1 && !visited[j]) {
                                                  stack.push(j);
                                                  visited[j] = true;
                                              }
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 85,
        "Domain": "Advanced",
        "Purpose": "BreadthFirstSearch",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BreadthFirstSearch.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 ADJACENCY-MATRIX.
                            05  ROW OCCURS 5 TIMES.
                                10  COLUMN PIC 9 OCCURS 5 TIMES.
                        01 QUEUE PIC 9(2) OCCURS 5 TIMES.
                        01 FRONT PIC 9 VALUE 0.
                        01 REAR PIC 9 VALUE 0.
                        01 START-NODE PIC 9 VALUE 1.
                        01 VISITED PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 I PIC 9.
                        01 J PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE 0 TO FRONT REAR
                            PERFORM UNTIL FRONT > REAR
                                MOVE QUEUE(FRONT) TO I
                                MOVE FRONT + 1 TO FRONT
                                MOVE 'Y' TO VISITED(I)
                                DISPLAY "Node visited: " I
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J > N
                                    IF ADJACENCY-MATRIX(I, J) = 1 AND VISITED(J) = 'N'
                                        ADD 1 TO REAR
                                        MOVE J TO QUEUE(REAR)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """import java.util.LinkedList;
                              import java.util.Queue;

                              public class BreadthFirstSearch {
                                  private static final int N = 5;
                                  private static int[][] adjacencyMatrix = new int[N + 1][N + 1];
                                  private static boolean[] visited = new boolean[N + 1];
                                  private static Queue<Integer> queue = new LinkedList<>();

                                  public static void main(String[] args) {
                                      int startNode = 1;

                                      // Initialize adjacency matrix (1-based indexing)
                                      // adjacencyMatrix[i][j] = 1 if there is an edge from i to j

                                      breadthFirstSearch(startNode);
                                  }

                                  private static void breadthFirstSearch(int startNode) {
                                      queue.add(startNode);
                                      visited[startNode] = true;

                                      while (!queue.isEmpty()) {
                                          int node = queue.poll();
                                          System.out.println("Node visited: " + node);

                                          for (int j = 1; j <= N; j++) {
                                              if (adjacencyMatrix[node][j] == 1 && !visited[j]) {
                                                  queue.add(j);
                                                  visited[j] = true;
                                              }
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 86,
        "Domain": "Advanced",
        "Purpose": "BellmanFordAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BellmanFordAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 DIST PIC 9(4) OCCURS 5 TIMES VALUE 9999.
                        01 EDGE-WEIGHT PIC 9(3) OCCURS 5 TIMES INDEXED BY I.
                        01 SOURCE PIC 9 VALUE 1.
                        01 I PIC 9.
                        01 J PIC 9.
                        01 K PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE 0 TO DIST(SOURCE)
                            PERFORM VARYING K FROM 1 BY 1 UNTIL K >= N
                                PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                    PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                        IF DIST(J) > DIST(I) + EDGE-WEIGHT(I, J)
                                            MOVE DIST(I) + EDGE-WEIGHT(I, J) TO DIST(J)
                                        END-IF
                                    END-PERFORM
                                END-PERFORM
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class BellmanFordAlgorithm {
                                  private static final int N = 5;
                                  private static final int INF = 9999;
                                  private static int[][] edgeWeight = new int[N + 1][N + 1];
                                  private static int[] dist = new int[N + 1];

                                  public static void main(String[] args) {
                                      int source = 1;

                                      // Initialize edge weights and distances
                                      // edgeWeight[i][j] = weight of edge from i to j
                                      // dist[i] = shortest distance from source to i

                                      bellmanFord(source);
                                  }

                                  private static void bellmanFord(int source) {
                                      for (int i = 1; i <= N; i++) {
                                          dist[i] = INF;
                                      }
                                      dist[source] = 0;

                                      for (int k = 1; k < N; k++) {
                                          for (int i = 1; i <= N; i++) {
                                              for (int j = 1; j <= N; j++) {
                                                  if (dist[j] > dist[i] + edgeWeight[i][j]) {
                                                      dist[j] = dist[i] + edgeWeight[i][j];
                                                  }
                                              }
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 87,
        "Domain": "Advanced",
        "Purpose": "FloydWarshallAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. FloydWarshallAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 DIST PIC 9(4) OCCURS 5 TIMES.
                        01 ADJACENCY-MATRIX.
                            05  ROW OCCURS 5 TIMES.
                                10  COLUMN PIC 9(4) OCCURS 5 TIMES.
                        01 I PIC 9.
                        01 J PIC 9.
                        01 K PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    MOVE ADJACENCY-MATRIX(I, J) TO DIST(I, J)
                                END-PERFORM
                            END-PERFORM
                            PERFORM VARYING K FROM 1 BY 1 UNTIL K <= N
                                PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                    PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                        IF DIST(I, K) + DIST(K, J) < DIST(I, J)
                                            MOVE DIST(I, K) + DIST(K, J) TO DIST(I, J)
                                        END-IF
                                    END-PERFORM
                                END-PERFORM
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class FloydWarshallAlgorithm {
                                  private static final int N = 5;
                                  private static final int INF = 9999;
                                  private static int[][] dist = new int[N + 1][N + 1];
                                  private static int[][] adjacencyMatrix = new int[N + 1][N + 1];

                                  public static void main(String[] args) {
                                      // Initialize adjacency matrix (1-based indexing)
                                      // adjacencyMatrix[i][j] = weight of edge from i to j

                                      floydWarshall();
                                  }

                                  private static void floydWarshall() {
                                      for (int i = 1; i <= N; i++) {
                                          for (int j = 1; j <= N; j++) {
                                              dist[i][j] = adjacencyMatrix[i][j];
                                          }
                                      }

                                      for (int k = 1; k <= N; k++) {
                                          for (int i = 1; i <= N; i++) {
                                              for (int j = 1; j <= N; j++) {
                                                  if (dist[i][k] + dist[k][j] < dist[i][j]) {
                                                      dist[i][j] = dist[i][k] + dist[k][j];
                                                  }
                                              }
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 88,
        "Domain": "Advanced",
        "Purpose": "AStarAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. AStarAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 START-NODE PIC 9 VALUE 1.
                        01 END-NODE PIC 9 VALUE 5.
                        01 H PIC 9(4) OCCURS 5 TIMES.
                        01 G PIC 9(4) OCCURS 5 TIMES VALUE 9999.
                        01 F PIC 9(4) OCCURS 5 TIMES VALUE 9999.
                        01 OPEN-SET PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 CLOSED-SET PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 PARENT PIC 9 OCCURS 5 TIMES VALUE 0.
                        01 I PIC 9.
                        01 J PIC 9.
                        01 MIN PIC 9(4) VALUE 9999.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE 0 TO G(START-NODE)
                            COMPUTE F(START-NODE) = G(START-NODE) + H(START-NODE)
                            ADD 1 TO I
                            MOVE 'Y' TO OPEN-SET(START-NODE)
                            PERFORM A-STAR-SEARCH
                            STOP RUN.

                        A-STAR-SEARCH.
                        IF I > 0
                            MOVE 0 TO MIN
                            PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                IF OPEN-SET(J) = 'Y' AND F(J) < MIN
                                    MOVE F(J) TO MIN
                                    MOVE J TO I
                                END-IF
                            END-PERFORM
                            IF I != 0
                                MOVE 'Y' TO CLOSED-SET(I)
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    IF OPEN-SET(J) = 'N' AND G(J) > G(I) + H(J)
                                        MOVE G(I) + H(J) TO G(J)
                                        COMPUTE F(J) = G(J) + H(J)
                                        MOVE I TO PARENT(J)
                                        MOVE 'Y' TO OPEN-SET(J)
                                    END-IF
                                END-PERFORM
                                PERFORM A-STAR-SEARCH
                            END-IF
                        END-IF.""",
        "Java Translation": """public class AStarAlgorithm {
                                  private static final int N = 5;
                                  private static final int INF = 9999;
                                  private static int[] h = new int[N + 1];
                                  private static int[] g = new int[N + 1];
                                  private static int[] f = new int[N + 1];
                                  private static char[] openSet = new char[N + 1];
                                  private static char[] closedSet = new char[N + 1];
                                  private static int[] parent = new int[N + 1];

                                  public static void main(String[] args) {
                                      int startNode = 1;
                                      int endNode = 5;

                                      // Initialize heuristic values (h) for each node
                                      // Initialize g and f values to infinity for all nodes

                                      g[startNode] = 0;
                                      f[startNode] = g[startNode] + h[startNode];
                                      openSet[startNode] = 'Y';

                                      aStarSearch();
                                  }

                                  private static void aStarSearch() {
                                      int i = 0;
                                      while (i > 0) {
                                          int min = INF;
                                          for (int j = 1; j <= N; j++) {
                                              if (openSet[j] == 'Y' && f[j] < min) {
                                                  min = f[j];
                                                  i = j;
                                              }
                                          }

                                          if (i != 0) {
                                              closedSet[i] = 'Y';
                                              for (int j = 1; j <= N; j++) {
                                                  if (openSet[j] == 'N' && g[j] > g[i] + h[j]) {
                                                      g[j] = g[i] + h[j];
                                                      f[j] = g[j] + h[j];
                                                      parent[j] = i;
                                                      openSet[j] = 'Y';
                                                  }
                                              }
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 89,
        "Domain": "Advanced",
        "Purpose": "PageRankAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. PageRankAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 DAMPING-FACTOR PIC 9V9 VALUE 0.85.
                        01 ITERATIONS PIC 9 VALUE 100.
                        01 PAGE-RANK PIC 9V9(4) OCCURS 5 TIMES VALUE 0.2.
                        01 OUTLINKS PIC 9 OCCURS 5 TIMES.
                        01 INLINKS PIC 9 OCCURS 5 TIMES.
                        01 TEMP PIC 9V9(4) OCCURS 5 TIMES.
                        01 I PIC 9.
                        01 J PIC 9.
                        01 SUM PIC 9V9(4).
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                MOVE 0 TO TEMP(I)
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    IF OUTLINKS(J) != 0
                                        COMPUTE TEMP(I) = TEMP(I) + PAGE-RANK(J) / OUTLINKS(J)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                COMPUTE PAGE-RANK(I) = (1 - DAMPING-FACTOR) / N + DAMPING-FACTOR * TEMP(I)
                            END-PERFORM
                            DISPLAY "Final PageRank:"
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                DISPLAY PAGE-RANK(I)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class PageRankAlgorithm {
                                  private static final int N = 5;
                                  private static final double DAMPING_FACTOR = 0.85;
                                  private static final int ITERATIONS = 100;
                                  private static double[] pageRank = new double[N + 1];
                                  private static int[] outlinks = new int[N + 1];
                                  private static int[] inlinks = new int[N + 1];
                                  private static double[] temp = new double[N + 1];

                                  public static void main(String[] args) {
                                      // Initialize outlinks and inlinks for each node
                                      // Calculate PageRank
                                      pageRankAlgorithm();
                                  }

                                  private static void pageRankAlgorithm() {
                                      for (int i = 1; i <= N; i++) {
                                          temp[i] = 0.0;
                                          for (int j = 1; j <= N; j++) {
                                              if (outlinks[j] != 0) {
                                                  temp[i] += pageRank[j] / outlinks[j];
                                              }
                                          }
                                      }

                                      for (int i = 1; i <= N; i++) {
                                          pageRank[i] = (1 - DAMPING_FACTOR) / N + DAMPING_FACTOR * temp[i];
                                      }

                                      // Perform ITERATIONS times
                                  }
                              }"""
    },
    {
        "ID": 90,
        "Domain": "Advanced",
        "Purpose": "KruskalAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. KruskalAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 MST-EDGES OCCURS 4 TIMES.
                            05  U PIC 9.
                            05  V PIC 9.
                            05  WEIGHT PIC 9.
                        01 EDGE-LIST OCCURS 4 TIMES.
                            05  U PIC 9.
                            05  V PIC 9.
                            05  WEIGHT PIC 9.
                        01 PARENT PIC 9 OCCURS 5 TIMES.
                        01 I PIC 9.
                        01 J PIC 9.
                        01 MIN-WEIGHT PIC 9.
                        01 MIN-EDGE PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE 0 TO MST-EDGES
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                MOVE I TO PARENT(I)
                            END-PERFORM
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N - 1
                                MOVE 9999 TO MIN-WEIGHT
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    IF PARENT(J) != J AND EDGE-LIST(J, 3) < MIN-WEIGHT
                                        MOVE EDGE-LIST(J, 3) TO MIN-WEIGHT
                                        MOVE J TO MIN-EDGE
                                    END-IF
                                END-PERFORM
                                MOVE MIN-WEIGHT TO MST-EDGES(I, 3)
                                MOVE EDGE-LIST(MIN-EDGE, 1) TO MST-EDGES(I, 1)
                                MOVE EDGE-LIST(MIN-EDGE, 2) TO MST-EDGES(I, 2)
                                MOVE PARENT(EDGE-LIST(MIN-EDGE, 2)) TO PARENT(EDGE-LIST(MIN-EDGE, 1))
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class KruskalAlgorithm {
                                  private static final int N = 5;
                                  private static int[][] mstEdges = new int[4][3];
                                  private static int[][] edgeList = new int[4][3];
                                  private static int[] parent = new int[N + 1];

                                  public static void main(String[] args) {
                                      // Initialize edgeList with edges and weights
                                      // Sort edgeList by weights

                                      kruskalAlgorithm();
                                  }

                                  private static void kruskalAlgorithm() {
                                      for (int i = 1; i <= N; i++) {
                                          parent[i] = i;
                                      }

                                      for (int i = 1; i <= N - 1; i++) {
                                          int minWeight = 9999;
                                          int minEdge = 0;
                                          for (int j = 1; j <= N; j++) {
                                              if (parent[j] != j && edgeList[j][2] < minWeight) {
                                                  minWeight = edgeList[j][2];
                                                  minEdge = j;
                                              }
                                          }
                                          mstEdges[i][3] = minWeight;
                                          mstEdges[i][1] = edgeList[minEdge][1];
                                          mstEdges[i][2] = edgeList[minEdge][2];
                                          parent[edgeList[minEdge][2]] = parent[edgeList[minEdge][1]];
                                      }
                                  }
                              }"""
    },
    {
        "ID": 91,
        "Domain": "Advanced",
        "Purpose": "DijkstraAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. DijkstraAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 DISTANCE PIC 9(4) OCCURS 5 TIMES VALUE 9999.
                        01 VISITED PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 EDGE-WEIGHT PIC 9(3) OCCURS 5 TIMES INDEXED BY I.
                        01 START-NODE PIC 9 VALUE 1.
                        01 END-NODE PIC 9 VALUE 5.
                        01 MIN PIC 9(4) VALUE 9999.
                        01 I PIC 9.
                        01 J PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE 0 TO DISTANCE(START-NODE)
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                MOVE 'N' TO VISITED(I)
                            END-PERFORM
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                MOVE 9999 TO MIN
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    IF VISITED(J) = 'N' AND DISTANCE(J) < MIN
                                        MOVE J TO I
                                        MOVE DISTANCE(J) TO MIN
                                    END-IF
                                END-PERFORM
                                MOVE 'Y' TO VISITED(I)
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    IF VISITED(J) = 'N' AND DISTANCE(I) + EDGE-WEIGHT(I, J) < DISTANCE(J)
                                        COMPUTE DISTANCE(J) = DISTANCE(I) + EDGE-WEIGHT(I, J)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class DijkstraAlgorithm {
                                  private static final int N = 5;
                                  private static final int INF = 9999;
                                  private static int[][] edgeWeight = new int[N + 1][N + 1];
                                  private static int[] distance = new int[N + 1];
                                  private static char[] visited = new char[N + 1];

                                  public static void main(String[] args) {
                                      int startNode = 1;
                                      int endNode = 5;

                                      // Initialize edge weights and distances
                                      // edgeWeight[i][j] = weight of edge from i to j
                                      // distance[i] = shortest distance from startNode to i

                                      dijkstraAlgorithm(startNode);
                                  }

                                  private static void dijkstraAlgorithm(int startNode) {
                                      for (int i = 1; i <= N; i++) {
                                          distance[i] = INF;
                                          visited[i] = 'N';
                                      }
                                      distance[startNode] = 0;

                                      for (int i = 1; i <= N; i++) {
                                          int min = INF;
                                          int minNode = -1;
                                          for (int j = 1; j <= N; j++) {
                                              if (visited[j] == 'N' && distance[j] < min) {
                                                  min = distance[j];
                                                  minNode = j;
                                              }
                                          }

                                          if (minNode == -1) {
                                              break;
                                          }

                                          visited[minNode] = 'Y';

                                          for (int j = 1; j <= N; j++) {
                                              if (visited[j] == 'N' && distance[minNode] + edgeWeight[minNode][j] < distance[j]) {
                                                  distance[j] = distance[minNode] + edgeWeight[minNode][j];
                                              }
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 92,
        "Domain": "Advanced",
        "Purpose": "DepthFirstSearchAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. DepthFirstSearchAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 ADJACENCY-MATRIX.
                            05  ROW OCCURS 5 TIMES.
                                10  COLUMN PIC 9 OCCURS 5 TIMES.
                        01 VISITED PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 STACK PIC 9(4) OCCURS 5 TIMES.
                        01 TOP PIC 9 VALUE 0.
                        01 START-NODE PIC 9 VALUE 1.
                        01 I PIC 9.
                        01 J PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                MOVE 'N' TO VISITED(I)
                            END-PERFORM
                            MOVE START-NODE TO STACK
                            MOVE 1 TO TOP
                            PERFORM UNTIL TOP = 0
                                MOVE STACK(TOP) TO I
                                IF VISITED(I) = 'N'
                                    DISPLAY I
                                    MOVE 'Y' TO VISITED(I)
                                END-IF
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    IF ADJACENCY-MATRIX(I, J) = 1 AND VISITED(J) = 'N'
                                        ADD 1 TO TOP
                                        MOVE J TO STACK(TOP)
                                    END-IF
                                END-PERFORM
                                SUBTRACT 1 FROM TOP
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class DepthFirstSearchAlgorithm {
                                  private static final int N = 5;
                                  private static int[][] adjacencyMatrix = new int[5][5];
                                  private static char[] visited = new char[N + 1];
                                  private static int[] stack = new int[N + 1];
                                  private static int top = 0;
                                  private static int startNode = 1;

                                  public static void main(String[] args) {
                                      // Initialize adjacency matrix
                                      // Initialize visited array
                                      // Call depthFirstSearch(startNode);
                                  }

                                  private static void depthFirstSearch(int startNode) {
                                      stack[++top] = startNode;
                                      visited[startNode] = 'Y';

                                      while (top != 0) {
                                          int i = stack[top];
                                          boolean found = false;
                                          for (int j = 1; j <= N; j++) {
                                              if (adjacencyMatrix[i][j] == 1 && visited[j] == 'N') {
                                                  stack[++top] = j;
                                                  visited[j] = 'Y';
                                                  found = true;
                                                  break;
                                              }
                                          }
                                          if (!found) {
                                              System.out.println(stack[top--]);
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 93,
        "Domain": "Advanced",
        "Purpose": "BreadthFirstSearchAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BreadthFirstSearchAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 ADJACENCY-MATRIX.
                            05  ROW OCCURS 5 TIMES.
                                10  COLUMN PIC 9 OCCURS 5 TIMES.
                        01 VISITED PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 QUEUE PIC 9(4) OCCURS 5 TIMES.
                        01 FRONT PIC 9 VALUE 0.
                        01 REAR PIC 9 VALUE 0.
                        01 START-NODE PIC 9 VALUE 1.
                        01 I PIC 9.
                        01 J PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                MOVE 'N' TO VISITED(I)
                            END-PERFORM
                            MOVE START-NODE TO REAR
                            MOVE START-NODE TO FRONT
                            MOVE 'Y' TO VISITED(START-NODE)
                            PERFORM UNTIL FRONT > REAR
                                ADD 1 TO FRONT
                                MOVE QUEUE(FRONT) TO I
                                DISPLAY I
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J <= N
                                    IF ADJACENCY-MATRIX(I, J) = 1 AND VISITED(J) = 'N'
                                        ADD 1 TO REAR
                                        MOVE J TO QUEUE(REAR)
                                        MOVE 'Y' TO VISITED(J)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class BreadthFirstSearchAlgorithm {
                                  private static final int N = 5;
                                  private static int[][] adjacencyMatrix = new int[5][5];
                                  private static char[] visited = new char[N + 1];
                                  private static int[] queue = new int[N + 1];
                                  private static int front = 0;
                                  private static int rear = 0;
                                  private static int startNode = 1;

                                  public static void main(String[] args) {
                                      // Initialize adjacency matrix
                                      // Initialize visited array
                                      // Call breadthFirstSearch(startNode);
                                  }

                                  private static void breadthFirstSearch(int startNode) {
                                      queue[++rear] = startNode;
                                      visited[startNode] = 'Y';

                                      while (front <= rear) {
                                          int i = queue[front++];
                                          System.out.println(i);

                                          for (int j = 1; j <= N; j++) {
                                              if (adjacencyMatrix[i][j] == 1 && visited[j] == 'N') {
                                                  queue[++rear] = j;
                                                  visited[j] = 'Y';
                                              }
                                          }
                                      }
                                  }
                              }"""
    },
    {
        "ID": 94,
        "Domain": "Advanced",
        "Purpose": "PrimAlgorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. PrimAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9 VALUE 5.
                        01 MST-EDGES OCCURS 4 TIMES.
                            05  U PIC 9.
                            05  V PIC 9.
                            05  WEIGHT PIC 9.
                        01 ADJACENCY-MATRIX.
                            05  ROW OCCURS 5 TIMES.
                                10  COLUMN PIC 9 OCCURS 5 TIMES.
                        01 VISITED PIC X OCCURS 5 TIMES VALUE 'N'.
                        01 NEAREST PIC 9 OCCURS 5 TIMES VALUE 0.
                        01 MIN-DISTANCE PIC 9 VALUE 9999.
                        01 MIN-U PIC 9.
                        01 MIN-V PIC 9.
                        01 I PIC 9.
                        01 J PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE 'Y' TO VISITED(1)
                            PERFORM VARYING I FROM 2 BY 1 UNTIL I <= N
                                MOVE ADJACENCY-MATRIX(1, I) TO NEAREST(I)
                                MOVE ADJACENCY-MATRIX(1, I) TO MIN-DISTANCE
                            END-PERFORM
                            PERFORM VARYING I FROM 2 BY 1 UNTIL I <= N
                                MOVE 9999 TO MIN-DISTANCE
                                PERFORM VARYING J FROM 2 BY 1 UNTIL J <= N
                                    IF VISITED(J) = 'N' AND NEAREST(J) < MIN-DISTANCE
                                        MOVE J TO MIN-V
                                        MOVE NEAREST(J) TO MIN-DISTANCE
                                    END-IF
                                END-PERFORM
                                MOVE 'Y' TO VISITED(MIN-V)
                                MOVE MIN-V TO MST-EDGES(I, 2)
                                MOVE NEAREST(MIN-V) TO MST-EDGES(I, 3)
                                PERFORM VARYING J FROM 2 BY 1 UNTIL J <= N
                                    IF VISITED(J) = 'N' AND ADJACENCY-MATRIX(MIN-V, J) < NEAREST(J)
                                        MOVE ADJACENCY-MATRIX(MIN-V, J) TO NEAREST(J)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class PrimAlgorithm {
                                  private static final int N = 5;
                                  private static int[][] mstEdges = new int[4][3];
                                  private static int[][] adjacencyMatrix = new int[5][5];
                                  private static char[] visited = new char[N + 1];
                                  private static int[] nearest = new int[N + 1];

                                  public static void main(String[] args) {
                                      // Initialize adjacency matrix
                                      // Call primAlgorithm();
                                  }

                                  private static void primAlgorithm() {
                                      visited[1] = 'Y';

                                      for (int i = 2; i <= N; i++) {
                                          nearest[i] = adjacencyMatrix[1][i];
                                      }

                                      for (int i = 2; i <= N; i++) {
                                          int minDistance = 9999;
                                          int minV = 0;
                                          for (int j = 2; j <= N; j++) {
                                              if (visited[j] == 'N' && nearest[j] < minDistance) {
                                                  minDistance = nearest[j];
                                                  minV = j;
                                              }
                                          }

                                          visited[minV] = 'Y';
                                          mstEdges[i][2] = minV;
                                          mstEdges[i][3] = nearest[minV];

                                          for (int j = 2; j <= N; j++) {
                                              if (visited[j] == 'N' && adjacencyMatrix[minV][j] < nearest[j]) {
                                                  nearest[j] = adjacencyMatrix[minV][j];
                                              }
                                          }
                                      }
                                  }
                              }"""
    }

]

# file path for the CSV file
csv_file = "cobol_java_pairs.csv"


# Write to the CSV file
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["ID", "Domain", "Purpose", "COBOL Code", "Java Translation"])
    writer.writeheader()
    for pair in cobol_java_pairs:
        writer.writerow(pair)

print(f"CSV file '{csv_file}' has been created successfully.")


import pandas as pd
df=pd.read_csv("/content/cobol_java_pairs.csv")
df

# Tokenize the code by splitting on whitespace and symbols
def tokenize_code(code):
    tokens = re.findall(r"[\w]+|[\S]", code)
    return tokens

# Remove single-line comments starting with "!" and multi-line comments enclosed in "*> ... <*"
def remove_comments(code):
    code = re.sub(r"!.*$", "", code, flags=re.MULTILINE)
    code = re.sub(r"\*>.*?<\*", "", code, flags=re.DOTALL)
    return code

# Normalize identifiers to lowercase
def normalize_identifiers(code):
    code = code.lower()
    return code

# Remove comments from the code, Tokenize the code, Normalize identifiers, Join tokens back into a single string
def preprocess_code(code):
    code = remove_comments(code)
    tokens = tokenize_code(code)
    code = normalize_identifiers(code)
    code = " ".join(tokens)
    return code


df = pd.read_csv("/content/cobol_java_pairs.csv")

df["Preprocessed COBOL Code"] = df["COBOL Code"].apply(preprocess_code)
df["Preprocessed Java Translation"] = df["Java Translation"].apply(preprocess_code)


# file path for the preprocessed CSV file and Write to the CSV file
csv_file_preprocessed = "cobol_java_pairs_preprocessed.csv"

df.to_csv(csv_file_preprocessed, index=False)
print(f"Preprocessed CSV file '{csv_file_preprocessed}' has been created successfully.")


import pandas as pd
df_preprocessed=pd.read_csv("/content/cobol_java_pairs_preprocessed.csv")
df_preprocessed


from keras.layers import Dropout, LSTM, Dense, Dropout, Embedding, Input
from keras.optimizers import Adam

# Tokenize COBOL code
cobol_input_sentences=df_preprocessed["Preprocessed COBOL Code"]
tokenizer_cobol = Tokenizer()
tokenizer_cobol.fit_on_texts(df_preprocessed["Preprocessed COBOL Code"])
encoder_input_sequences = tokenizer_cobol.texts_to_sequences(df_preprocessed["Preprocessed COBOL Code"])

print(cobol_input_sentences)
print("*************************************************************************************************************************************************************************")
print(encoder_input_sequences)


# Pad COBOL sequences
max_encoder_seq_length = max(len(seq) for seq in encoder_input_sequences)
encoder_input_data = pad_sequences(encoder_input_sequences, maxlen=max_encoder_seq_length, padding='post')

print(encoder_input_data)


# Get the number of unique tokens in COBOL vocabulary
num_encoder_tokens = len(tokenizer_cobol.word_index) + 1
print(num_encoder_tokens)


# Tokenize Java translation
java_input_sentences=df_preprocessed["Preprocessed Java Translation"]
tokenizer_java = Tokenizer()
tokenizer_java.fit_on_texts(df_preprocessed["Preprocessed Java Translation"])
decoder_input_sequences = tokenizer_java.texts_to_sequences(df_preprocessed["Preprocessed Java Translation"])

print(java_input_sentences)
print("*************************************************************************************************************************************************************************")
print(decoder_input_sequences)


# Pad Java sequences
max_decoder_seq_length = max(len(seq) for seq in decoder_input_sequences)
decoder_input_data = pad_sequences(decoder_input_sequences, maxlen=max_decoder_seq_length, padding='post')


# Get the number of unique tokens in Java vocabulary
num_decoder_tokens = len(tokenizer_java.word_index) + 1
print(num_decoder_tokens)


# Initialize decoder target data
decoder_target_data = np.zeros((len(decoder_input_sequences), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

# Populate decoder target data
for i, seq in enumerate(decoder_input_sequences):
    for t, token in enumerate(seq):
        decoder_target_data[i, t, token] = 1.0


from sklearn.model_selection import train_test_split
# Split the data into training and validation sets
encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val, decoder_target_train, decoder_target_val = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.2
)

latent_dim = 592
# Define the encoder input layer
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs1, state_h1, state_c1 = encoder_lstm1(encoder_embedding)

encoder_lstm2 = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm2(encoder_outputs1)
encoder_outputs = Dropout(0.5)(encoder_outputs)
encoder_states = [state_h, state_c]


# Adjust num_decoder_tokens to match the target data
num_decoder_tokens_target = len(tokenizer_java.word_index) + 1


# Define the decoder input layer
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs1, _, _ = decoder_lstm1(decoder_embedding, initial_state=encoder_states)

decoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs2, _, _ = decoder_lstm2(decoder_outputs1)
decoder_dense = Dense(num_decoder_tokens_target, activation="softmax")
decoder_outputs = Dropout(0.5)(decoder_outputs2)



# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


from keras.optimizers import Adam, RMSprop
optimizer1 = RMSprop(lr=0.001)


# Try different optimizers and learning rates
optimizer = RMSprop(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=optimizer1, loss="categorical_crossentropy", metrics=["accuracy"])



# Train the model
translation_model=model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_target_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_target_val),
    batch_size=128,
    epochs=10
)

CSV file 'cobol_java_pairs.csv' has been created successfully.
Preprocessed CSV file 'cobol_java_pairs_preprocessed.csv' has been created successfully.
0     IDENTIFICATION DIVISION . PROGRAM - ID . Hello...
1     IDENTIFICATION DIVISION . PROGRAM - ID . AddNu...
2     IDENTIFICATION DIVISION . PROGRAM - ID . Large...
3     IDENTIFICATION DIVISION . PROGRAM - ID . Facto...
4     IDENTIFICATION DIVISION . PROGRAM - ID . Fibon...
                            ...                        
68    IDENTIFICATION DIVISION . PROGRAM - ID . Krusk...
69    IDENTIFICATION DIVISION . PROGRAM - ID . Dijks...
70    IDENTIFICATION DIVISION . PROGRAM - ID . Depth...
71    IDENTIFICATION DIVISION . PROGRAM - ID . Bread...
72    IDENTIFICATION DIVISION . PROGRAM - ID . PrimA...
Name: Preprocessed COBOL Code, Length: 73, dtype: object
*************************************************************************************************************************************************************************
[[29,

Epoch 1/10
1/1 [==============================] - 68s 68s/step - loss: 2.2288 - accuracy: 3.1277e-04 - val_loss: 1.5843 - val_accuracy: 0.0048
Epoch 2/10
1/1 [==============================] - 61s 61s/step - loss: 2.4001 - accuracy: 0.0093 - val_loss: 1.4018 - val_accuracy: 0.0047
Epoch 3/10
1/1 [==============================] - 59s 59s/step - loss: 2.4094 - accuracy: 0.0098 - val_loss: 1.4481 - val_accuracy: 0.0057
Epoch 4/10
1/1 [==============================] - 56s 56s/step - loss: 2.2222 - accuracy: 0.0091 - val_loss: 1.2922 - val_accuracy: 0.0060
Epoch 5/10
1/1 [==============================] - 59s 59s/step - loss: 2.2827 - accuracy: 0.0078 - val_loss: 1.3217 - val_accuracy: 0.0050
Epoch 6/10
1/1 [==============================] - 56s 56s/step - loss: 2.2054 - accuracy: 0.0077 - val_loss: 1.2906 - val_accuracy: 0.0011
Epoch 7/10
1/1 [==============================] - 56s 56s/step - loss: 2.2963 - accuracy: 0.0061 - val_loss: 1.5081 - val_accuracy: 0.0032
Epoch 8/10
1/1 [=======

NameError: name 'cobol_code' is not defined

In [11]:
def preprocess_cobol_code(cobol_code):
    # Tokenize and pad the input sequence
    cobol_sequence = tokenizer_cobol.texts_to_sequences([cobol_code])
    cobol_padded = pad_sequences(cobol_sequence, padding='post')

    return cobol_padded


cobol_code="""IDENTIFICATION DIVISION.
                        PROGRAM-ID. HelloWorld.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Hello, World!".
                            STOP RUN."""
# Preprocess the COBOL code
preprocessed_cobol = preprocess_cobol_code(cobol_code)

# Create a dummy decoder input (assuming a single input sequence)
decoder_input = np.zeros((1, 1))

# Perform the translation
java_sequence = model.predict([preprocessed_cobol, decoder_input])

# Convert the predicted sequence to a list of integers
java_sequence_list = [int(i) for i in java_sequence.flatten()]

# Convert the predicted sequence back to text
java_translation = tokenizer_java.sequences_to_texts([java_sequence_list])[0]


# print(java_sequence)
print(java_sequence_list)
print("Predicted Java Translation:", java_translation)

1/1 [==============================] - 0s 67ms/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,